In [1]:
'''
Goal of the code is score a customers order based on their order experience on the criterias such as Pre Sub OOS%, OOS%, 
Delivery delay, Handoff delay, complaints, refunds and NPS scoring on different questions
'''

'\nGoal of the code is score a customers order based on their order experience on the criterias such as Pre Sub OOS%, OOS%, \nDelivery delay, Handoff delay, complaints, refunds and NPS scoring on different questions\n'

In [1]:
#import libraries
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime, timezone
from datetime import timedelta
import datetime as dt
import random

<html><h1>Data Collection and Validation </h1></html>

In [2]:
#Connection setup anf functions
class SFConnection:
    def __init__(self):
        self.name = ''
        self.connection = ''
        self.engine = ''

    def open_conn(self):
        username = self.name + '@safeway.com'

        self.engine = create_engine(URL(
        account = 'abs_itds_prd.west-us-2.privatelink',#'abs_itds_prd.west-us-2.azure',
        user = username,
        password = '',
        database = 'EDM_BIZOPS_PRD',
        schema = 'FBI',
        warehouse = 'PROD_CORP_SVCS_WH',
        role='ROLE_USERGROUP_FBI_ALL',
        authenticator='externalbrowser'
        ))
        self.connection = self.engine.connect().execution_options(autocommit=False)
        self.connection.begin()
        print("Connection Opened")

    def test_conn(self):
        try:
            self.open_conn()
            results = self.connection.execute('select current_version()').fetchone()
            print("Snowflake connected to Version: {}".format(results[0]))
            return("CONNECTED")
        except Exception as e:
            print("Snowflake connection Error")
            return("ERR")

    def run_query(self, sql):
        self.connection.execute("BEGIN")
        self.connection.execute(sql)
        self.connection.execute("COMMIT")

    def query_to_df(self, sql):
        #results = self.connection.execute(sql)
        return pd.read_sql_query(sql, self.connection)
        #return df_txn
        
    def close_conn(self):
        self.connection.close()
        print("Connection Closed")

    def add_name(self, name):
        self.name = name
    
    #accepts a dataframe and a table name in lowercase. It is fixed currently to the FBI Schema. You only need to pass through the tablename
    #The df must be in the exact format as table
    #This will upload in chuncks of 10,000 rows to Snowflake
    #This method will replace all current values from Dataframe 
    def load_dataframe_REPLACE(self, inputdf, tablename):
        inputdf.to_sql(name=tablename.lower(), con=self.engine, if_exists='replace', index=False, chunksize=10000)
    
    #Method for inserting value without replacing to data tables 
    def load_dataframe_APPEND(self, inputdf, tablename):
        inputdf.to_sql(name=tablename.lower(), con=self.engine, if_exists='append', index=False, chunksize=10000)

In [3]:
#Setup Connection to Snowflake
conn = SFConnection()
conn.add_name("nacke08")
conn.test_conn()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Connection Opened
Snowflake connected to Version: 8.9.2


'CONNECTED'

In [4]:
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#LOGIC ADDED TO DETERMINE IF TABLES NEED TO HAVE DUPLCIATE VALUES REMOVED BEFORE AN UPDATE CAN CONTINUE
#Added: Nick 4/7/2023
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

current_date = dt.datetime.now().strftime("%Y-%m-%d")
# ExistingDatasql = 'SELECT * FROM EDM_BIZOPS_PRD.FBI.ESA_PROACTIVE_COMMUNICATION_ANALYSIS_OUTPUT WHERE UPDATE_DATE = date(\'' + current_date + '\')'
# dfExistingData = conn.query_to_df(ExistingDatasql)
# #If results are populated then remove this data from the table. 
# #This will be completed for the archive table
# if len(dfExistingData) != 0: 
#     conn.run_query('DELETE FROM EDM_BIZOPS_PRD.FBI.ESA_PROACTIVE_COMMUNICATION_ANALYSIS_OUTPUT WHERE UPDATE_DATE = date(\'' + current_date + '\')')


In [5]:
FRACTAL_HH_TABLE  = '''
INSERT overwrite INTO EDM_BIZOPS_PRD.FBI.FRACTAL_MIR_HH_DATA
(   
	with txn_data as (
	select ORDER_STATUS_NM,
	ORDER_ID,
	ORDER_DT,
	DELIVERY_TYPE,
	SVCLVL_TYPE,
	STYPE,
	REGISTER_NBR,
	TXN_DTE,
	WEEK_ID,
	TZ,
	SLOT_START,
	SLOT_END,
	DIVISION,
	HOUSEHOLD_ID,
	CUSTOMER_SUBSCRIPTION_IND,
	DELIVERY_FEE_AMT,
	ORDER_TOTAL_AMT,
	NET_SALES,
	UNITS,
	GROSS_SALES,
	MKDN_AMT,
	TAX_AMT,
	TOTAL_MISC_AMT,
	ORDERED,
	OOS,
	SUBS,
	(OOS+SUBS)/iff(UNITS=0,1,UNITS) as Pre_Sub_OOS,
	PERFECT_ORDER,
	BAD_SUB_FLAG,
	BAD_SUB_ITEMS,
	OOS_FLAG,
	SUB_FLAG,
	DELIV_TS,
	CARRIER_TYPE,
	CARRIER,
	OTD,
	OTD_ELIGIBLE,
	WAIT_TIME_IN_MINS,
	OTH_ELIGIBLE,
	OTH5,
	PROMO_CODE_FLAG,
	PROMO_CODES,
	FIRST_TYPE_PROMO_CODE_FLAG,
	--SURVEY_DATE,
	--SURVEY_Q1_SCORE,
	--CC_DATE,
	--CC_COMMENT_TYPE,
	CC_DESCRPTION_1,
	MISSING_ITEMS_FLAG,
	FRESH_PASS_HHS,
	--FRESHPASS_SUBSCRIPBED_DT,
	J4U_HHS,
	J4U_FIRST_VISIT_DT
	from "EDM_BIZOPS_PRD"."FBI"."NA_FRACTAL_ORDER_TXN_MASTER_CUBE_HISTORY" 
 	where txn_dte >= '{txn_start_date}' and txn_dte <= '{txn_end_date}'
 	AND ORDERED > 0
),
household_ids_list as (
	select distinct household_id
	from txn_data where household_id is not NULL
),
HDR_TXN_1 as (
	select txn_dte,txn_id,register_nbr,store_id,card_nbr,net_amt from "EDM_VIEWS_PRD"."DW_EDW_VIEWS"."TXN_HDR" 
),
HDR_TXN_2 as (
	select b.household_id,
	a.txn_dte,
	a.txn_id,
	a.register_nbr,
	a.store_id,
	a.card_nbr,
	sum(a.net_amt) as net_sales
	from HDR_TXN_1 a
	join "EDM_VIEWS_PRD"."DW_VIEWS"."SMV_RETAIL_CUSTOMER_LOYALTY_PROGRAM_HOUSEHOLD" b
	on a.card_nbr = b.LOYALTY_PROGRAM_CARD_NBR
	group by 1,2,3,4,5,6
),
HDR_TXN_3 as (
	select a.household_id,
	a.txn_dte,
	a.txn_id,
	a.register_nbr,
	a.net_sales,
	b.banner_nm
	from HDR_TXN_2 a
	join "EDM_VIEWS_PRD"."DW_BIZOPS_VIEWS"."LU_STORE_FINANCE_OM" b
	on a.store_id = b.store_id
),
HDR_TXN_ids as (
	select * from HDR_TXN_3
	where household_id is not NULL 
	--and banner_nm = 'SAFEWAY' 
	and household_id in (select * from household_ids_list)
),
HDR_TXN_dates as (
	select household_id,date(min(txn_dte)) as First_txn_dte,date(max(txn_dte)) as Last_txn_dte from HDR_TXN_ids where register_nbr IN (99,173) group by 1
),
HDR_TXN_dates_flags as (
	select household_id,
	(case when First_txn_dte>='{six_months}' and First_txn_dte<'{txn_start_date}' then 1 else 0 end) as Acquired_within_6month,
	(case when First_txn_dte<'{six_months}' then 1 else 0 end) as Acquired_prior_6month
	from HDR_TXN_dates
),
HDR_TXN_counts as (
	select household_id,
	sum(case when register_nbr in (99,173) then 1 else 0 end) as count_ecom_txn,
	sum(case when register_nbr not in (99,173) then 1 else 0 end) as count_instore_txn
	from HDR_TXN_ids
	where txn_dte>='{six_months}' and txn_dte<'{txn_start_date}'
	group by 1
),
BNC_ECOM AS (
	select HOUSEHOLD_ID, min(txn_dte) as First_Ecom_Txn_Dte
	from HDR_TXN_ids where register_nbr IN (99,173)
	group by HOUSEHOLD_ID
),
BNC_ACI AS (
	select HOUSEHOLD_ID, min(txn_dte) as First_Txn_Dte
	from HDR_TXN_ids
	group by HOUSEHOLD_ID
),
--for BNC to ECom, filter is already applied for ecom txns
BNC_ECOM_flag_table AS (
	select HOUSEHOLD_ID, min(txn_dte) as BNC_to_Ecom_date
	from HDR_TXN_ids where register_nbr IN (99,173) and txn_dte<'{txn_start_date}'
	group by HOUSEHOLD_ID
),
BNC_ACI_flag_table AS (
	select HOUSEHOLD_ID, min(txn_dte) as BNC_to_ACI_date
	from HDR_TXN_ids where txn_dte<'{txn_start_date}'
	group by HOUSEHOLD_ID
),
RECENT_ECOM_TXN AS (
	SELECT HOUSEHOLD_ID, MAX(TXN_DTE) AS RECENT_ECOM_TXN_DTE
	FROM HDR_TXN_ids WHERE TXN_DTE < '{txn_start_date}' AND register_nbr IN (99,173)
	GROUP BY HOUSEHOLD_ID
),
Date_Weeks as (
	select week_id,d_date
	from "EDM_VIEWS_PRD"."DW_EDW_VIEWS"."LU_DAY_MERGE"
	group by 1,2
),
RECENT_ECOM_TXN_MAPPED AS (
	select a.HOUSEHOLD_ID, b.week_id as RECENT_ECOM_TXN_Week
	from RECENT_ECOM_TXN a
	left join Date_Weeks b
	on a.RECENT_ECOM_TXN_DTE = b.d_date
),
RECENT_INSTORE_TXN AS (
	SELECT HOUSEHOLD_ID, MAX(TXN_DTE) AS RECENT_INSTORE_TXN_DTE
	FROM HDR_TXN_ids WHERE TXN_DTE < '{txn_start_date}' AND register_nbr NOT IN (99,173)
	GROUP BY HOUSEHOLD_ID
),
RECENT_INSTORE_TXN_MAPPED AS (
	select a.HOUSEHOLD_ID, b.week_id as RECENT_INSTORE_TXN_Week
	from RECENT_INSTORE_TXN a
	left join Date_Weeks b
	on a.RECENT_INSTORE_TXN_DTE = b.d_date
),
segment_latest as (
	select HOUSEHOLD_ID, max(UPDATE_TS_MT) as UPDATE_TS_MT from "EDM_VIEWS_PRD"."DW_BIZOPS_VIEWS"."WKLY_HH_SEG" where UPDATE_TS_MT<'{txn_start_date}' group by 1
),
segment_latest_facts as (
	select a.HOUSEHOLD_ID, max(b.FACTS_SEGMENT_NAME) as FACTS_SEGMENT_NAME
	from segment_latest a, "EDM_VIEWS_PRD"."DW_BIZOPS_VIEWS"."WKLY_HH_SEG" b
	where a.HOUSEHOLD_ID = b.HOUSEHOLD_ID and a.UPDATE_TS_MT = b.UPDATE_TS_MT
	group by 1
),
HDR_combined as (
	select a.*,
	b.count_ecom_txn as count_ecom_txn_last6month,
	b.count_instore_txn as count_instore_txn_last6month,
	c.RECENT_ECOM_TXN_Week as latest_ecom_txn_week,
	d.RECENT_INSTORE_TXN_Week as latest_instore_txn_week,
	e.First_Ecom_Txn_Dte,
	f.First_Txn_Dte,
	g.FACTS_SEGMENT_NAME,
	iff(h.BNC_to_Ecom_date IS NULL,1,0) as BNC_to_eCom_flag,
	iff(i.BNC_to_ACI_date IS NULL,1,0) as BNC_to_ACI_flag,
	j.RECENT_ECOM_TXN_DTE as LATEST_ECOM_TXN_DTE,
	k.RECENT_INSTORE_TXN_DTE AS LATEST_INSTORE_TXN_DTE                
	from HDR_TXN_dates_flags a
	left join HDR_TXN_counts b
	on a.household_id = b.household_id
	left join RECENT_ECOM_TXN_MAPPED c
	on a.household_id = c.household_id
	left join RECENT_INSTORE_TXN_MAPPED d
	on a.household_id = d.household_id
	left join BNC_ECOM e
	on a.household_id = e.household_id
	left join BNC_ACI f
	on a.household_id = f.household_id
	left join segment_latest_facts g
	on a.household_id = g.household_id
	left join BNC_ECOM_flag_table h
	on a.household_id = h.household_id
	left join BNC_ACI_flag_table i
	on a.household_id = i.household_id
	left join RECENT_ECOM_TXN j
	on a.household_id = j.household_id
	left join RECENT_INSTORE_TXN k
	on a.household_id = k.household_id                 
),
final_data_date_lookup as (
	select a.*, 
	b.week_id as FIRST_ECOM_TXN_WEEK,
	c.week_id as FIRST_TXN_WEEK
	from HDR_combined a
	left join Date_Weeks b
	on a.FIRST_ECOM_TXN_DTE = b.d_date
	left join Date_Weeks c
	on a.FIRST_TXN_DTE = c.d_date
)
SELECT
* 
from final_data_date_lookup
);'''.format(txn_start_date = '2024-03-04', txn_end_date = '2024-03-04', six_months = '2023-09-06')
conn.run_query(FRACTAL_HH_TABLE)

In [6]:
#Fetching transaction level data
#This will have to be repalced at somepoint with the new ESA tables once they are validated
#remove all txns from households that are currently in the control group
#txn_data_sql = 'SELECT * FROM EDM_BIZOPS_PRD.FBI.FRACTAL_MIR_TXN_DATA_NPS_VIEW where txn_dte >= \'2023-10-08\' and txn_dte <= \'2023-10-10\' and ((banner = \'Jewel-Osco\') or (household_id in (\'990072161305\', \'990059087520\', \'990075616372\', \'932046858249\', \'990053617034\')))'\
txn_data_sql = 'SELECT * FROM EDM_BIZOPS_PRD.FBI.FRACTAL_MIR_TXN_DATA_NPS_VIEW where txn_dte >= \'2024-03-04\' and txn_dte <= \'2024-03-04\''
df_txns = conn.query_to_df(txn_data_sql)
#Fetching household data
#This will have to be repalced at somepoint with the new ESA tables once they are validated
hhs_data_sql = 'SELECT * FROM EDM_BIZOPS_PRD.FBI.FRACTAL_MIR_HH_DATA'
df_hhs = conn.query_to_df(hhs_data_sql)
#Fetching offers data
sqlThird = 'SELECT * FROM EDM_BIZOPS_PRD.FBI.ESA_MIR_ECOMM_RETENTION_OFFERS'
df_offers = conn.query_to_df(sqlThird)
#segmentation file
sqlFourth = 'SELECT * FROM EDM_BIZOPS_PRD.FBI.ESA_PROACTIVE_SEGMENTATION_IMPLEMENTATION_TESTING'#'SELECT * FROM EDM_BIZOPS_PRD.FBI.ESA_PROACTIVE_SEGMENTATION_IMPLEMENTATION'
segmentation = conn.query_to_df(sqlFourth)
#Repeat Customers (Updated 8/8/23)
#Could make this Redeem Date in the future for further optimization. Currently similar to MIR
sqlRepeat = "SELECT DISTINCT HHS_ID FROM EDM_BIZOPS_PRD.FBI.ESA_PROACTIVE_COMMUNICATION_PBI WHERE EMAIL_DATE > (CURRENT_DATE-31) AND TEST_EXPERIENCE IN ('$10 off', '$5 off')"
RepeatCustomers = conn.query_to_df(sqlRepeat)
conn.close_conn()

Connection Closed


In [7]:
df_txns.count()
df_txns["txn_dte"].unique()
df_hhs.count()
df_txns.head()

,order_status_nm,version_id,order_id,order_dt,channel_type,delivery_type,svclvl_type,stype,register_nbr,txn_id,...,NPS_CHKBOX_Product_Findability_Category_Difficulty,NPS_CHKBOX_Product_Findability_Reason,NPS_CHKBOX_Pick_Up_Reason,NPS_CHKBOX_Delivery_Reason,NPS_CHKBOX_Pick_Up_Problem_Reason,NPS_CHKBOX_Delivery_Problem_Reason,om_area,om,district_nm,fiscal_week_id
0,COMPLETED,1,83953855,2024-03-03,APP,DUG,STANDARD,DUG,99,6897014058,...,None,None,None,None,None,None,SoCal 01,Billy Bright,Central Coast,202402
1,COMPLETED,1,83987413,2024-03-04,APP,DELIVERY,STANDARD,2HR,99,6897931104,...,None,None,None,None,None,None,SoCal 06,Tony Meyer,Ctrl Orange Cty,202402
2,COMPLETED,2,83960516,2024-03-03,APP,DELIVERY,STANDARD,1HR,99,6895496577,...,None,None,None,None,None,None,Seattle 10,Tonya Lane,Cntrl Washingto,202402
3,COMPLETED,2,83944402,2024-03-03,SITE,DELIVERY,STANDARD,1HR,99,6896236560,...,None,None,None,None,None,None,SoCal 05,Joe Bisera,Ctrl Orange Cty,202402
4,COMPLETED,1,83981764,2024-03-04,APP,DELIVERY,EXPRESS,1HR,99,6896133551,...,None,None,None,None,None,None,Southwest 5,Bill Gaddis,Stuc/Country/Sc,202402


In [8]:
df_txns.count()
df_txns.columns
df_txns.groupby(["order_status_nm", "version_id"]).count()

order_id  order_dt  channel_type  delivery_type  \
order_status_nm version_id                                                    
COMPLETED       1              68589     68589         68589          68585   
                2               4905      4905          4905           4905   
                3               1311      1311          1311           1311   
                4                452       452           452            452   
                5                163       163           163            163   
                6                 80        80            80             80   
                7                 41        41            41             41   
                8                 21        21            21             21   
                9                 11        11            11             11   
                10                 2         2             2              2   
                11                 3         3             3              3   
                12                 2         2             2              2   
                13                 1         1             1              1   
                14                 2         2             2              2   

                            svclvl_type  stype  register_nbr  txn_id  txn_dte  \
order_status_nm version_id                                                      
COMPLETED       1                 68589  68585         68589   68589    68589   
                2                  4905   4905          4905    4905     4905   
                3                  1311   1311          1311    1311     1311   
                4                   452    452           452     452      452   
                5                   163    163           163     163      163   
                6                    80     80            80      80       80   
                7                    41     41            41      41       41   
                8                    21     21            21      21       21   
                9                    11     11            11      11       11   
                10                    2      2             2       2        2   
                11                    3      3             3       3        3   
                12                    2      2             2       2        2   
                13                    1      1             1       1        1   
                14                    2      2             2       2        2   

                            week_id  ...  \
order_status_nm version_id           ...   
COMPLETED       1             68589  ...   
                2              4905  ...   
                3              1311  ...   
                4               452  ...   
                5               163  ...   
                6                80  ...   
                7                41  ...   
                8                21  ...   
                9                11  ...   
                10                2  ...   
                11                3  ...   
                12                2  ...   
                13                1  ...   
                14                2  ...   

                            NPS_CHKBOX_Product_Findability_Category_Difficulty  \
order_status_nm version_id                                                       
COMPLETED       1                                                        1370    
                2                                                         143    
                3                                                          42    
                4                                                          16    
                5                                                           6    
                6                                                           4    
                7                                                           1    
                8      

In [9]:
#Get only the created and completed orders
df_created_orders = df_txns[(df_txns["order_status_nm"]== "CREATED") | 
                           (df_txns["order_status_nm"]== "COMPLETED")].reset_index(drop = True)
df_created_orders[df_created_orders["txn_dte"].isna()].head()

,order_status_nm,version_id,order_id,order_dt,channel_type,delivery_type,svclvl_type,stype,register_nbr,txn_id,...,NPS_CHKBOX_Product_Findability_Category_Difficulty,NPS_CHKBOX_Product_Findability_Reason,NPS_CHKBOX_Pick_Up_Reason,NPS_CHKBOX_Delivery_Reason,NPS_CHKBOX_Pick_Up_Problem_Reason,NPS_CHKBOX_Delivery_Problem_Reason,om_area,om,district_nm,fiscal_week_id


In [10]:
#Getting the required columns only
df_orders = df_created_orders[['order_status_nm', 'version_id', 'order_id', 'order_dt', 'channel_type',
       'delivery_type', 'svclvl_type', 'stype', 'txn_id',
       'txn_dte', 'week_id', 'order_ts_local', 'txn_tm', 'tz', 'slot_start',
       'slot_end', 'mfc_flag', 'store_id', 'division', 'banner',
       'store_city_nm', 'store_state_id', 'store_zip5_id', 'urbanicity',
       'card_nbr', 'household_id','delivery_fee_amt', 'order_total_amt',
       'net_sales', 'units', 'gross_sales', 'mkdn_amt', 'tax_amt',
       'total_misc_amt', 'aiv', 'mkdn_per', 'ordered', 'oos', 'subs', 'perfect_order', 'bad_sub_flag', 'bad_sub_items',
       'oos_flag', 'sub_flag', 'till_by_time', 'till_tm', 'ott', 'ott_eligible',
       'deliv_ts', 'otd', 'otd_eligible', 'wait_time_in_mins', 'oth_eligible', 'oth5','total_refund_amt', 'refund_count', 'refund_reasons',
       'comment_count', 'comment_types', 'comments', 'survey_id', 'order_id_nps', 'NPS_Order_Timeliness', 'NPS_Order_Accuracy', 'NPS_Item_Availability',
       'NPS_Ease_of_Pick-up', 'NPS_Ease_of_Ordering_Process',
       'NPS_Associate_Friendliness', 'NPS_Substitution_Satisfaction',
       'NPS_Substitution_Dissatisfaction_Reason',
       'NPS_Likelihood_to_Recommend', 'NPS_Reason_for_Score', 'nps_survey_type_nm','nps_survey_mobile_device_ind',
       'nps_device_brand_nm','nps_device_operating_system_nm','nps_survey_banner_nm','nps_survey_channel_type_nm',
       'nps_survey_response_ts','NPS_Safety_Precautions_Taken','NPS_Product_Quality','NPS_Product_Freshness', 'fresh_pass_hhs',
       'carrier_type', 'carrier'
]]

<h3>Removing refunds via Customer rejected substitutions </h3>

In [11]:
#Marking refunds order under is_refund for flag
df_orders["is_refund"] = 0
df_orders.loc[df_orders["total_refund_amt"].isna() == False ,"is_refund"] = 1
df_orders.groupby('is_refund')['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\833530516.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders["is_refund"] = 0


is_refund
0    71215
1     4368
Name: order_id, dtype: int64

In [12]:
#Unflag refund orders that are under customer rejected substitutions
df_orders.loc[df_orders['refund_reasons'] == 'Customer rejected substitutions', 'is_refund'] = 0.0

<html> <h3> Accuracy and Completeness Issue </h3> </html>

In [13]:
#Calculate OOS percent
df_orders["OOS_percent"] = (df_orders["oos"]/df_orders["ordered"])*100
df_orders[["OOS_percent", "oos", "ordered"]]

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\1314163345.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders["OOS_percent"] = (df_orders["oos"]/df_orders["ordered"])*100


,OOS_percent,oos,ordered
0,0.000000,0.0,8.0
1,13.333333,2.0,15.0
2,0.000000,0.0,11.0
3,3.030303,1.0,33.0
4,10.000000,1.0,10.0
...,...,...,...
75578,9.523810,2.0,21.0
75579,0.000000,0.0,8.0
75580,0.000000,0.0,41.0
75581,20.000000,1.0,5.0


In [14]:
#Fill the empty values in OOS percent with 1000
df_orders["OOS_percent"] = df_orders["OOS_percent"].fillna(1000)
df_orders[df_orders["OOS_percent"].isna()]["OOS_percent"]

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\3664474846.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders["OOS_percent"] = df_orders["OOS_percent"].fillna(1000)


Series([], Name: OOS_percent, dtype: float64)

In [15]:
#Calculate SUBS percent
df_orders["Subs_percent"] = (df_orders["subs"]/df_orders["ordered"])*100
df_orders[["Subs_percent", "subs", "ordered"]]

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2595499238.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders["Subs_percent"] = (df_orders["subs"]/df_orders["ordered"])*100


,Subs_percent,subs,ordered
0,37.500000,3.0,8.0
1,0.000000,0.0,15.0
2,9.090909,1.0,11.0
3,3.030303,1.0,33.0
4,0.000000,0.0,10.0
...,...,...,...
75578,0.000000,0.0,21.0
75579,0.000000,0.0,8.0
75580,7.317073,3.0,41.0
75581,0.000000,0.0,5.0


In [16]:
#Fill the empty values in SUBS percent with 1000
df_orders["Subs_percent"] = df_orders["Subs_percent"].fillna(1000)
df_orders[df_orders["Subs_percent"].isna()]["Subs_percent"]

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\998802270.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders["Subs_percent"] = df_orders["Subs_percent"].fillna(1000)


Series([], Name: Subs_percent, dtype: float64)

In [17]:
#Calculating pre-sub OOS
df_orders["PRE_SUB_OOS"] = df_orders["oos"] + df_orders["subs"]

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\1085425993.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders["PRE_SUB_OOS"] = df_orders["oos"] + df_orders["subs"]


In [18]:
#Calculating pre-sub OOS percent
df_orders["Pre_Subs_OOS_Per"] = (df_orders["PRE_SUB_OOS"]/df_orders["ordered"])*100
df_orders[["Pre_Subs_OOS_Per", "PRE_SUB_OOS", "ordered"]]

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\1602849141.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders["Pre_Subs_OOS_Per"] = (df_orders["PRE_SUB_OOS"]/df_orders["ordered"])*100


,Pre_Subs_OOS_Per,PRE_SUB_OOS,ordered
0,37.500000,3.0,8.0
1,13.333333,2.0,15.0
2,9.090909,1.0,11.0
3,6.060606,2.0,33.0
4,10.000000,1.0,10.0
...,...,...,...
75578,9.523810,2.0,21.0
75579,0.000000,0.0,8.0
75580,7.317073,3.0,41.0
75581,20.000000,1.0,5.0


In [19]:
#Filling N/A values in presub OOS percent with 1000
df_orders["Pre_Subs_OOS_Per"] = df_orders["Pre_Subs_OOS_Per"].fillna(1000)
df_orders[df_orders["Pre_Subs_OOS_Per"].isna()]["Pre_Subs_OOS_Per"]

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\869656887.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders["Pre_Subs_OOS_Per"] = df_orders["Pre_Subs_OOS_Per"].fillna(1000)


Series([], Name: Pre_Subs_OOS_Per, dtype: float64)

In [20]:
'''
Thresholds used for Post-Sub OOS scoring
0% - 1 - Great Job
>0% - <15% - 2 - Average
>=15% - <=100% - 3 - Needs Improvement
Data not available - 0
'''
df_orders['SCORE_OOS_per'] = 0
df_orders.loc[(df_orders["OOS_percent"] == 0) , "SCORE_OOS_per"] = 1
#df_orders.loc[(df_orders["OOS_percent"] >= 0) &
#                      (df_orders["OOS_percent"] < 1), "SCORE_OOS_per"] = 1
df_orders.loc[(df_orders["OOS_percent"] > 0) &
                      (df_orders["OOS_percent"] < 15), "SCORE_OOS_per"] = 2
df_orders.loc[(df_orders["OOS_percent"] >= 15) &
                      (df_orders["OOS_percent"] <= 100), "SCORE_OOS_per"] = 3

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\797887297.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['SCORE_OOS_per'] = 0


In [21]:
'''
Thresholds used for Pre-Sub OOS scoring
0% - <5%: 1 : Great Job
>=5% - <25% : 2 : Average
>=25% - <=100% : 3 : Needs Improvement
Data not available - 0
'''
df_orders['SCORE_PreSub_OOS_per'] = 0
df_orders.loc[(df_orders["Pre_Subs_OOS_Per"] >= 0) &
                      (df_orders["Pre_Subs_OOS_Per"] < 5), "SCORE_PreSub_OOS_per"] = 1
df_orders.loc[(df_orders["Pre_Subs_OOS_Per"] >= 5) &
                      (df_orders["Pre_Subs_OOS_Per"] < 25), "SCORE_PreSub_OOS_per"] = 2
df_orders.loc[(df_orders["Pre_Subs_OOS_Per"] >= 25) &
                      (df_orders["Pre_Subs_OOS_Per"] <= 100), "SCORE_PreSub_OOS_per"] = 3
df_orders.groupby("SCORE_PreSub_OOS_per")['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\793627069.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['SCORE_PreSub_OOS_per'] = 0


SCORE_PreSub_OOS_per
0      283
1    31363
2    36929
3     7008
Name: order_id, dtype: int64

In [22]:
df_orders['NPS_Order_Accuracy'].fillna(1000, inplace=True)
df_orders['NPS_Substitution_Satisfaction'].fillna(1000, inplace=True)

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\731802618.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['NPS_Order_Accuracy'].fillna(1000, inplace=True)
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\731802618.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['NPS_Substitution_Satisfaction'].fillna(1000, inplace=True)


In [23]:
'''
Thresholds used for NPS Order Accuracy scoring
9 - 10: 1 : Great Job
7 - 8 : 2 : Average
0 - 6 : 3 : Needs Improvement
NULL : 0 : Data not available
'''
df_orders['SCORE_NPS_Order_Accuracy'] = 0
df_orders.loc[(df_orders["NPS_Order_Accuracy"] >= 9) &
                      (df_orders["NPS_Order_Accuracy"] <= 10), "SCORE_NPS_Order_Accuracy"] = 1
df_orders.loc[(df_orders["NPS_Order_Accuracy"] >= 7) &
                      (df_orders["NPS_Order_Accuracy"] <= 8), "SCORE_NPS_Order_Accuracy"] = 2
df_orders.loc[(df_orders["NPS_Order_Accuracy"] >= 0) &
                      (df_orders["NPS_Order_Accuracy"] <= 6), "SCORE_NPS_Order_Accuracy"] = 3
df_orders.groupby("SCORE_NPS_Order_Accuracy")['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\3176689654.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['SCORE_NPS_Order_Accuracy'] = 0


SCORE_NPS_Order_Accuracy
0    74054
1     1000
2      232
3      297
Name: order_id, dtype: int64

In [24]:
'''
Thresholds used for NPS Substitution Satisfaction scoring
9 - 10: 1 : Great Job
7 - 8 : 2 : Average
0 - 6 : 3 : Needs Improvement
NULL : 0 : Data not available
'''
df_orders['SCORE_NPS_Substitution_Satisfaction'] = 0
df_orders.loc[(df_orders["NPS_Substitution_Satisfaction"] >= 9) &
                      (df_orders["NPS_Substitution_Satisfaction"] <= 10), "SCORE_NPS_Substitution_Satisfaction"] = 1
df_orders.loc[(df_orders["NPS_Substitution_Satisfaction"] >= 7) &
                      (df_orders["NPS_Substitution_Satisfaction"] <= 8), "SCORE_NPS_Substitution_Satisfaction"] = 2
df_orders.loc[(df_orders["NPS_Substitution_Satisfaction"] >= 0) &
                      (df_orders["NPS_Substitution_Satisfaction"] <= 6), "SCORE_NPS_Substitution_Satisfaction"] = 3
df_orders.groupby("SCORE_NPS_Substitution_Satisfaction")['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\3017215500.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['SCORE_NPS_Substitution_Satisfaction'] = 0


SCORE_NPS_Substitution_Satisfaction
0    74847
1      429
2      105
3      202
Name: order_id, dtype: int64

In [25]:
#Getting worst score among the NPS order accuracy questions
df_orders['Order_Accuracy_NPS'] = df_orders[['SCORE_NPS_Order_Accuracy', 'SCORE_NPS_Substitution_Satisfaction']].max(axis=1)
#Getting worst score among the order accuracy operational metrics
df_orders['Order_Accuracy_Operational'] = df_orders[['SCORE_OOS_per', 'SCORE_PreSub_OOS_per']].max(axis=1)

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2979628600.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['Order_Accuracy_NPS'] = df_orders[['SCORE_NPS_Order_Accuracy', 'SCORE_NPS_Substitution_Satisfaction']].max(axis=1)
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2979628600.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['Order_Accuracy_Operational'] = df_orders[['SCORE_OOS_per', 'SCORE_PreSub_OOS_per']].max(axis=1)


In [26]:
#Assigning final Order Accuracy score based on the decided conditions (find details in source of truth file)
df_orders['Order_Accuracy_Final'] = 0
df_orders.loc[(df_orders['Order_Accuracy_NPS'] == 1) & 
              (df_orders['Order_Accuracy_Operational'] == 1), 'Order_Accuracy_Final'] = 1
df_orders.loc[(df_orders['Order_Accuracy_NPS'] == 1) & 
              ((df_orders['Order_Accuracy_Operational'] == 2) | (df_orders['Order_Accuracy_Operational'] == 3)), 
              'Order_Accuracy_Final'] = 2
df_orders.loc[(df_orders['Order_Accuracy_NPS'] == 2) & 
              (df_orders['Order_Accuracy_Operational'] == 3), 'Order_Accuracy_Final'] = 3
df_orders.loc[(df_orders['Order_Accuracy_NPS'] == 2) & 
              ((df_orders['Order_Accuracy_Operational'] == 1) | (df_orders['Order_Accuracy_Operational'] == 2)), 
              'Order_Accuracy_Final'] = 2
df_orders.loc[(df_orders['Order_Accuracy_NPS'] == 3) & 
              ((df_orders['Order_Accuracy_Operational'] == 1) | (df_orders['Order_Accuracy_Operational'] == 2) |
              (df_orders['Order_Accuracy_Operational'] == 3)), 
              'Order_Accuracy_Final'] = 3
df_orders.loc[(df_orders['Order_Accuracy_NPS'] == 0), 'Order_Accuracy_Final'] = df_orders['Order_Accuracy_Operational']

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2576795381.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['Order_Accuracy_Final'] = 0


In [27]:
df_orders[['order_id', 'household_id', 'oos', 'OOS_percent', 'subs', 'Subs_percent', 'PRE_SUB_OOS', 'Pre_Subs_OOS_Per',
          'NPS_Order_Accuracy', 'NPS_Substitution_Satisfaction', 'Order_Accuracy_NPS',
       'Order_Accuracy_Operational', 'Order_Accuracy_Final']]

,order_id,household_id,oos,OOS_percent,subs,Subs_percent,PRE_SUB_OOS,Pre_Subs_OOS_Per,NPS_Order_Accuracy,NPS_Substitution_Satisfaction,Order_Accuracy_NPS,Order_Accuracy_Operational,Order_Accuracy_Final
0,83953855,NaN,0.0,0.000000,3.0,37.500000,3.0,37.500000,1000.0,1000.0,0,3,3
1,83987413,NaN,2.0,13.333333,0.0,0.000000,2.0,13.333333,1000.0,1000.0,0,2,2
2,83960516,NaN,0.0,0.000000,1.0,9.090909,1.0,9.090909,1000.0,1000.0,0,2,2
3,83944402,NaN,1.0,3.030303,1.0,3.030303,2.0,6.060606,1000.0,1000.0,0,2,2
4,83981764,NaN,1.0,10.000000,0.0,0.000000,1.0,10.000000,1000.0,1000.0,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75578,84008360,NaN,2.0,9.523810,0.0,0.000000,2.0,9.523810,1000.0,1000.0,0,2,2
75579,84001403,NaN,0.0,0.000000,0.0,0.000000,0.0,0.000000,10.0,1000.0,1,1,1
75580,83954748,NaN,0.0,0.000000,3.0,7.317073,3.0,7.317073,1000.0,1000.0,0,2,2
75581,84025924,NaN,1.0,20.000000,0.0,0.000000,1.0,20.000000,1000.0,1000.0,0,3,3


In [28]:
#Changing score for accurary related refunds order
order_accuracy_refunds = ['Compensation - Missing Item', 'Compensation - Out of Stock']
df_orders['Order_Accuracy_Refunds'] = 0
df_orders.loc[(df_orders['is_refund'] == 1.0) & (df_orders['refund_reasons'].isin(order_accuracy_refunds)), 
              'Order_Accuracy_Refunds'] = 3
df_orders.groupby(['Order_Accuracy_Refunds'])['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2975601349.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['Order_Accuracy_Refunds'] = 0


Order_Accuracy_Refunds
0    75502
3       81
Name: order_id, dtype: int64

In [29]:
#Splitting refunds reasons with more than 1 refunds on order to traverse through and identify accuracy refunds
df_orders['refunds_split'] = 'Not Applicable'
for i in range(0, len(df_orders)):
    if((df_orders["is_refund"][i] == 1) and (df_orders["refund_count"][i] > 1)):
        #print(df_orders["comments"][i].split(' - '))
        df_orders['refunds_split'][i] = df_orders["refund_reasons"][i].split(' :: ')

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\1651478805.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['refunds_split'] = 'Not Applicable'
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\1651478805.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['refunds_split'][i] = df_orders["refund_reasons"][i].split(' :: ')


In [30]:
df_orders[df_orders['refund_count'] > 1]['refunds_split']

222                     [Product Related, Product Related]
547                                      [Product Related]
626                     [Product Related, Product Related]
745                                                     []
1098                                     [Product Related]
                               ...                        
74250    [Late - 2 Hours Or More, Late - 2 Hours Or Mor...
74515    [Late - 0 - 30 Minutes, Late - 0 - 30 Minutes,...
74923                                    [Product Related]
74979                                                   []
75457    [Late - 2 Hours Or More, Late - 2 Hours Or Mor...
Name: refunds_split, Length: 210, dtype: object

In [31]:
for i in range(0, len(df_orders)):
    if((df_orders["is_refund"][i] == 1) and (df_orders["refund_count"][i] > 1)):
        for x in order_accuracy_refunds:
            if x in df_orders['refunds_split'][i]:
                df_orders["Order_Accuracy_Refunds"][i] = 3       
df_orders.groupby(['Order_Accuracy_Refunds'])['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\3315529129.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders["Order_Accuracy_Refunds"][i] = 3


Order_Accuracy_Refunds
0    75481
3      102
Name: order_id, dtype: int64

In [32]:
#Splitting complaints
df_orders['is_complaint'] = 0
df_orders.loc[df_orders['comment_count'] > 0.0, 'is_complaint'] = 1
df_orders.groupby('is_complaint')['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\3250445623.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['is_complaint'] = 0


is_complaint
0    74150
1     1433
Name: order_id, dtype: int64

In [33]:
#Modifying score for order accuracy related complaints
order_accuracy_complaints = ['Missing Items', 'Wrong Substitution(s)', 'Out of Stock', 'Wrong Weight/Quantity',
                            'Requested No Substitution(s)', 'Wrong Item Received', 'Overcharge', 'More expensive',
                            'Delivery Fee', 'Requested Substitutions', 'Wrong Weight/Quantity']
df_orders['Order_Accuracy_Complaints'] = 0
df_orders.loc[(df_orders['is_complaint'] == 1.0) & (df_orders['comments'].isin(order_accuracy_complaints)), 
              'Order_Accuracy_Complaints'] = 3
df_orders.groupby(['Order_Accuracy_Complaints'])['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\665484015.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['Order_Accuracy_Complaints'] = 0


Order_Accuracy_Complaints
0    75313
3      270
Name: order_id, dtype: int64

In [34]:
df_orders['comments_split'] = 'Not Applicable'
for i in range(0, len(df_orders)):
    if((df_orders["is_complaint"][i] == 1) and (df_orders["comment_count"][i] > 1)):
        #print(df_orders["comments"][i].split(' - '))
        df_orders['comments_split'][i] = df_orders["comments"][i].split(' - ')

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\3943473661.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['comments_split'] = 'Not Applicable'
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\3943473661.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['comments_split'][i] = df_orders["comments"][i].split(' - ')


In [35]:
df_orders[df_orders['comment_count'] > 1]['comments_split']

222        [Out of Date/Quality, Address Change/Selection]
547                         [Missing Items, Missing Items]
626                         [Missing Items, Missing Items]
1098     [Wrong Weight/Quantity, System Unavailable, Co...
1324            [Wrong Item Received, Wrong Item Received]
                               ...                        
74678           [By Customer, By Customer, Confirm Refund]
74923                        [Confirm Slot, Missing Items]
75134                         [Confirm Slot, Confirm Slot]
75221                       [Attitude/Rude, Attitude/Rude]
75225                           [By Customer, By Customer]
Name: comments_split, Length: 325, dtype: object

In [36]:
for i in range(0, len(df_orders)):
    if((df_orders["is_complaint"][i] == 1) and (df_orders["comment_count"][i] > 1)):
        for x in order_accuracy_complaints:
            if x in df_orders['comments_split'][i]:
                df_orders["Order_Accuracy_Complaints"][i] = 3       
df_orders.groupby(['Order_Accuracy_Complaints'])['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\951586766.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders["Order_Accuracy_Complaints"][i] = 3


Order_Accuracy_Complaints
0    75188
3      395
Name: order_id, dtype: int64

In [37]:
#Assigning final score after including complaints and refunds
df_orders['Order_Accuracy_Final_inc_C_R'] = df_orders[['Order_Accuracy_Final', 'Order_Accuracy_Refunds', 'Order_Accuracy_Complaints']].max(axis=1)

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\4170597625.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['Order_Accuracy_Final_inc_C_R'] = df_orders[['Order_Accuracy_Final', 'Order_Accuracy_Refunds', 'Order_Accuracy_Complaints']].max(axis=1)


<html> <h3> Delivery Issue </h3> </html>

In [38]:
df_orders[(df_orders["delivery_type"] == "DELIVERY")][["order_id","delivery_type","deliv_ts"]]

,order_id,delivery_type,deliv_ts
1,83987413,DELIVERY,2024-03-04 14:18:41.021
2,83960516,DELIVERY,2024-03-04 09:02:20.862
3,83944402,DELIVERY,2024-03-04 10:01:43.659
4,83981764,DELIVERY,2024-03-04 12:29:14.499
17,83988558,DELIVERY,2024-03-04 17:12:42.878
...,...,...,...
75572,83957638,DELIVERY,2024-03-04 10:43:22.458
75574,83439123,DELIVERY,2024-03-04 18:24:06.855
75575,83990521,DELIVERY,2024-03-04 16:23:01.341
75576,83947007,DELIVERY,2024-03-04 11:59:51.276


In [39]:
#Calculate delivery difference
df_orders.loc[(df_orders["delivery_type"] == "DELIVERY") & 
                      (df_orders["deliv_ts"] > df_orders["slot_end"]), "Delivery_Late"] = 1
df_orders.loc[(df_orders["delivery_type"] == "DELIVERY") & 
                      (df_orders["deliv_ts"] > df_orders["slot_end"]), "Delivery_Diff"] = df_orders["deliv_ts"] - df_orders["slot_end"]
df_orders["Delivery_Late"].fillna(0, inplace=True)
df_orders.loc[(df_orders["delivery_type"] == "DELIVERY") & (df_orders["deliv_ts"].isna()), "Delivery_Late"] = 1000
#df_created_orders.groupby("Delivery_Diff").count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\420183222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders.loc[(df_orders["delivery_type"] == "DELIVERY") &
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\420183222.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders.loc[(df_orders["delivery_type"] == "DELIVERY") &
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\420183222.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

In [40]:
# #Calculate delivery difference
# #Modifcation Needed to account for early delivery
# df_orders.loc[(df_orders["delivery_type"] == "DELIVERY") & 
#                       ((df_orders["deliv_ts"] > df_orders["slot_end"]) | (df_orders["deliv_ts"] < df_orders["slot_start"])), "Delivery_Late"] = 1
# #Late Delivery
# df_orders.loc[(df_orders["delivery_type"] == "DELIVERY") & 
#                       (df_orders["deliv_ts"] > df_orders["slot_end"]), "Delivery_Diff"] = df_orders["deliv_ts"] - df_orders["slot_end"]
# #Early Delivery
# df_orders.loc[(df_orders["delivery_type"] == "DELIVERY") & 
#                       (df_orders["deliv_ts"] < df_orders["slot_start"]), "Delivery_Diff"] = df_orders["slot_start"] - df_orders["deliv_ts"]
# df_orders["Delivery_Late"].fillna(0, inplace=True)
# df_orders.loc[(df_orders["delivery_type"] == "DELIVERY") & (df_orders["deliv_ts"].isna()), "Delivery_Late"] = 1000
# #df_created_orders.groupby("Delivery_Diff").count()

In [41]:
df_orders["Delivery_delay_in_mins"] = 0.0
for i in range(0, len(df_orders)):
    if df_orders["Delivery_Late"][i] == 1.0:
        df_orders["Delivery_delay_in_mins"][i] = (df_orders["Delivery_Diff"][i].total_seconds())/60.0
    elif df_orders["Delivery_Late"][i] == 1000.0:
        df_orders["Delivery_delay_in_mins"][i] = -1000.0
    else:
        df_orders["Delivery_delay_in_mins"][i] = 0.0
df_orders.groupby("Delivery_delay_in_mins")["order_id"].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\672799436.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders["Delivery_delay_in_mins"] = 0.0
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\672799436.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders["Delivery_delay_in_mins"][i] = 0.0
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\672799436.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

Delivery_delay_in_mins
-1000.000000      271
 0.000000       73368
 0.000583           1
 0.031650           1
 0.051067           1
                ...  
 2218.995767        1
 2598.752850        1
 2771.775417        1
 3005.669883        1
 3251.280450        1
Name: order_id, Length: 1944, dtype: int64

In [42]:
'''
Need to add early delivery condition
Thresholds used for Delivery Delay on Delivery orders
0 - <1: 1 : Great Job
>=1 - <15 : 2 : Average
>=15 - max delivery time : 3 : Needs Improvement
0 - Data not available
'''
df_orders['SCORE_OTD'] = 0
df_orders.loc[(df_orders['Delivery_Late'] == 1) & ((df_orders["Delivery_delay_in_mins"] >= 0) &
                      (df_orders["Delivery_delay_in_mins"] < 1)), "SCORE_OTD"] = 1
df_orders.loc[(df_orders['Delivery_Late'] == 0) & (df_orders["delivery_type"] == 'DELIVERY'), "SCORE_OTD"] = 1
df_orders.loc[(df_orders['Delivery_Late'] == 1) & ((df_orders["Delivery_delay_in_mins"] >= 1) &
                      (df_orders["Delivery_delay_in_mins"] < 15)), "SCORE_OTD"] = 2
df_orders.loc[(df_orders['Delivery_Late'] == 1) & (df_orders["Delivery_delay_in_mins"] >= 15), "SCORE_OTD"] = 3

df_orders.groupby("SCORE_OTD").count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2314700792.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['SCORE_OTD'] = 0


,order_status_nm,version_id,order_id,order_dt,channel_type,delivery_type,svclvl_type,stype,txn_id,txn_dte,...,Order_Accuracy_Final,Order_Accuracy_Refunds,refunds_split,is_complaint,Order_Accuracy_Complaints,comments_split,Order_Accuracy_Final_inc_C_R,Delivery_Late,Delivery_Diff,Delivery_delay_in_mins
SCORE_OTD,,,,,,,,,,,,,,,,,,,,,
0,51249,51249,51249,51249,51249,51245,51249,51245,51249,51249,...,51249,51249,51249,51249,51249,51249,51249,51249,0,51249
1,22460,22460,22460,22460,22460,22460,22460,22460,22460,22460,...,22460,22460,22460,22460,22460,22460,22460,22460,70,22460
2,677,677,677,677,677,677,677,677,677,677,...,677,677,677,677,677,677,677,677,677,677
3,1197,1197,1197,1197,1197,1197,1197,1197,1197,1197,...,1197,1197,1197,1197,1197,1197,1197,1197,1197,1197


In [43]:
#Calculating tilling delay for DUG orders only
df_orders["Tilling_delay_in_mins"] = 0.0
for i in range(0, len(df_orders)):
    if df_orders["delivery_type"][i] == "DUG":
        #df_orders["Tilling_delay_in_mins"][i] = ((df_orders["txn_tm"][i] - df_orders["slot_start"][i]).total_seconds())/60.0
        df_orders["Tilling_delay_in_mins"][i] = ((df_orders["till_tm"][i] - df_orders["till_by_time"][i]).total_seconds())/60.0
    else:
        df_orders["Tilling_delay_in_mins"][i] = 0.0
#df_created_orders.groupby("Delivery_Diff").count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\554335570.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders["Tilling_delay_in_mins"] = 0.0
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\554335570.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders["Tilling_delay_in_mins"][i] = ((df_orders["till_tm"][i] - df_orders["till_by_time"][i]).total_seconds())/60.0
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\554335570.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in t

In [44]:
df_orders[df_orders["Tilling_delay_in_mins"] >= 15.0]["order_id"]

18       83873322
19       83914900
30       84006235
42       83980069
43       83987420
           ...   
75490    83999124
75503    83963927
75505    83865946
75518    83975729
75546    84009539
Name: order_id, Length: 5844, dtype: int64

In [45]:
df_orders["Tilling_Late"] = 0
for i in range(0, len(df_orders)):
    if df_orders["Tilling_delay_in_mins"][i] > 0.0:
        df_orders["Tilling_Late"][i] = 1
    else:
        df_orders["Tilling_Late"][i] = 0

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2364870801.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders["Tilling_Late"] = 0
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2364870801.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders["Tilling_Late"][i] = 0
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2364870801.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [46]:
'''
Thresholds used for Tilling Delay on DUG orders
0 - 0: 1 : Great Job
>0 - <15 : 2 : Average
>=15 - max tilling delay : 3 : Needs Improvement
'''
df_orders.loc[(df_orders['Tilling_Late'] == 0) & (df_orders["delivery_type"] == 'DUG'), "SCORE_Tilling"] = 1
df_orders.loc[(df_orders['Tilling_Late'] == 1) & ((df_orders["Tilling_delay_in_mins"] > 0) &
                      (df_orders["Tilling_delay_in_mins"] < 15)), "SCORE_Tilling"] = 2
df_orders.loc[(df_orders['Tilling_Late'] == 1) & (df_orders["Tilling_delay_in_mins"] >= 15), "SCORE_Tilling"] = 3

df_orders.groupby("SCORE_Tilling").count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\937024445.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders.loc[(df_orders['Tilling_Late'] == 0) & (df_orders["delivery_type"] == 'DUG'), "SCORE_Tilling"] = 1


,order_status_nm,version_id,order_id,order_dt,channel_type,delivery_type,svclvl_type,stype,txn_id,txn_dte,...,is_complaint,Order_Accuracy_Complaints,comments_split,Order_Accuracy_Final_inc_C_R,Delivery_Late,Delivery_Diff,Delivery_delay_in_mins,SCORE_OTD,Tilling_delay_in_mins,Tilling_Late
SCORE_Tilling,,,,,,,,,,,,,,,,,,,,,
1.0,42341,42341,42341,42341,42341,42341,42341,42341,42341,42341,...,42341,42341,42341,42341,42341,0,42341,42341,42341,42341
2.0,2789,2789,2789,2789,2789,2789,2789,2789,2789,2789,...,2789,2789,2789,2789,2789,0,2789,2789,2789,2789
3.0,5844,5844,5844,5844,5844,5844,5844,5844,5844,5844,...,5844,5844,5844,5844,5844,0,5844,5844,5844,5844


In [47]:
'''
Thresholds used for Wait_time_in_mins on DUG orders
0 - <5: 1 : Great Job
>=5 - <10 : 2 : Average
>=10 - <=35 : 3 : Needs Improvement
0 - Data not available or negative wait time
'''
df_orders.loc[(df_orders["delivery_type"] == 'DUG') & (df_orders["wait_time_in_mins"].isna() == True), "SCORE_OTH"] = 0
df_orders.loc[(df_orders["delivery_type"] == 'DUG') & (df_orders["wait_time_in_mins"] < 0), "SCORE_OTH"] = 0
df_orders.loc[(df_orders["delivery_type"] == 'DUG') & ((df_orders["wait_time_in_mins"] >= 0) &
                      (df_orders["wait_time_in_mins"] < 5)), "SCORE_OTH"] = 1
df_orders.loc[(df_orders["delivery_type"] == 'DUG') & ((df_orders["wait_time_in_mins"] >= 5) &
                      (df_orders["wait_time_in_mins"] < 10)), "SCORE_OTH"] = 2
df_orders.loc[(df_orders["delivery_type"] == 'DUG') & ((df_orders["wait_time_in_mins"] >= 10)), "SCORE_OTH"] = 3

df_orders.groupby("SCORE_OTH").count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\1951247322.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders.loc[(df_orders["delivery_type"] == 'DUG') & (df_orders["wait_time_in_mins"].isna() == True), "SCORE_OTH"] = 0


,order_status_nm,version_id,order_id,order_dt,channel_type,delivery_type,svclvl_type,stype,txn_id,txn_dte,...,Order_Accuracy_Complaints,comments_split,Order_Accuracy_Final_inc_C_R,Delivery_Late,Delivery_Diff,Delivery_delay_in_mins,SCORE_OTD,Tilling_delay_in_mins,Tilling_Late,SCORE_Tilling
SCORE_OTH,,,,,,,,,,,,,,,,,,,,,
0.0,5326,5326,5326,5326,5326,5326,5326,5326,5326,5326,...,5326,5326,5326,5326,0,5326,5326,5326,5326,5326
1.0,34512,34512,34512,34512,34512,34512,34512,34512,34512,34512,...,34512,34512,34512,34512,0,34512,34512,34512,34512,34512
2.0,9215,9215,9215,9215,9215,9215,9215,9215,9215,9215,...,9215,9215,9215,9215,0,9215,9215,9215,9215,9215
3.0,1921,1921,1921,1921,1921,1921,1921,1921,1921,1921,...,1921,1921,1921,1921,0,1921,1921,1921,1921,1921


In [48]:
#Getting order deilvery operational score for Delivery orders and DUG orders based on delivery delay, Tilling Delay and wait_time
df_orders['Order_Delivery_Operational'] = 0
for i in range(0, len(df_orders)):
    if(df_orders['delivery_type'][i] == 'DELIVERY'):
        df_orders['Order_Delivery_Operational'][i] = df_orders['SCORE_OTD'][i]
    else:
        if(pd.isna(df_orders['SCORE_OTH'][i]) == True):
            df_orders['Order_Delivery_Operational'][i] = df_orders['SCORE_Tilling'][i]
        else:
            df_orders['Order_Delivery_Operational'][i] = max(df_orders['SCORE_OTH'][i], df_orders['SCORE_Tilling'][i])

df_orders.groupby('Order_Delivery_Operational')['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2673251494.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['Order_Delivery_Operational'] = 0
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2673251494.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['Order_Delivery_Operational'][i] = max(df_orders['SCORE_OTH'][i], df_orders['SCORE_Tilling'][i])
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2673251494.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the doc

Order_Delivery_Operational
0.0      271
1.0    55821
2.0    10817
3.0     8670
Name: order_id, dtype: int64

In [49]:
'''
Thresholds used for NPS Delivery scoring
9 - 10: 1 : Great Job
7 - 8 : 2 : Average
0 - 6 : 3 : Needs Improvement
'''
df_orders['SCORE_NPS_Order_Delivery'] = 0
df_orders.loc[(df_orders["NPS_Order_Timeliness"] >= 9) &
                      (df_orders["NPS_Order_Timeliness"] <= 10), "SCORE_NPS_Order_Delivery"] = 1
df_orders.loc[(df_orders["NPS_Order_Timeliness"] >= 7) &
                      (df_orders["NPS_Order_Timeliness"] <= 8), "SCORE_NPS_Order_Delivery"] = 2
df_orders.loc[(df_orders["NPS_Order_Timeliness"] >= 0) &
                      (df_orders["NPS_Order_Timeliness"] <= 6), "SCORE_NPS_Order_Delivery"] = 3
df_orders.groupby("SCORE_NPS_Order_Delivery")['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\4192185882.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['SCORE_NPS_Order_Delivery'] = 0


SCORE_NPS_Order_Delivery
0    74032
1     1236
2       81
3      234
Name: order_id, dtype: int64

In [50]:
#Assigning score for order timeliness based on operational and NPS questions for order delivery (condtions in source of truth file)
df_orders['Order_Delivery_Final'] = 0
df_orders.loc[(df_orders['SCORE_NPS_Order_Delivery'] == 1) & 
              (df_orders['Order_Delivery_Operational'] == 1), 'Order_Delivery_Final'] = 1
df_orders.loc[(df_orders['SCORE_NPS_Order_Delivery'] == 1) & 
              ((df_orders['Order_Delivery_Operational'] == 2) | (df_orders['Order_Delivery_Operational'] == 3)), 
              'Order_Delivery_Final'] = 2
df_orders.loc[(df_orders['SCORE_NPS_Order_Delivery'] == 2) & 
              (df_orders['Order_Delivery_Operational'] == 3), 'Order_Delivery_Final'] = 3
df_orders.loc[(df_orders['SCORE_NPS_Order_Delivery'] == 2) & 
              ((df_orders['Order_Delivery_Operational'] == 1) | (df_orders['Order_Delivery_Operational'] == 2)), 
              'Order_Delivery_Final'] = 2
df_orders.loc[(df_orders['SCORE_NPS_Order_Delivery'] == 3) & 
              ((df_orders['Order_Delivery_Operational'] == 1) | (df_orders['Order_Delivery_Operational'] == 2) |
              (df_orders['Order_Delivery_Operational'] == 3)), 
              'Order_Delivery_Final'] = 3
df_orders.loc[(df_orders['SCORE_NPS_Order_Delivery'] == 0), 'Order_Delivery_Final'] = df_orders['Order_Delivery_Operational']

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\3579523339.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['Order_Delivery_Final'] = 0
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\3579523339.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders.loc[(df_orders['SCORE_NPS_Order_Delivery'] == 0), 'Order_Delivery_Final'] = df_orders['Order_Delivery_Operational']


In [51]:
#Modifying score using the delivery related refunds
order_delivery_refunds = ['Late - 0 - 30 Minutes', 'Late - 2 Hours Or More','Late - 30 Minutes - 2 Hours',
                          'Late Delivery - Cancelled & Refunded', 'Overcharged - Delivery Fee', 'Too Early']
df_orders['Order_Delivery_Refunds'] = 0
df_orders.loc[(df_orders['is_refund'] == 1.0) & (df_orders['refund_reasons'].isin(order_delivery_refunds)), 
              'Order_Delivery_Refunds'] = 3
df_orders.groupby(['Order_Delivery_Refunds'])['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2791962334.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['Order_Delivery_Refunds'] = 0


Order_Delivery_Refunds
0    75576
3        7
Name: order_id, dtype: int64

In [52]:
for i in range(0, len(df_orders)):
    if((df_orders["is_refund"][i] == 1) and (df_orders["refund_count"][i] > 1)):
        for x in order_delivery_refunds:
            if x in df_orders['refunds_split'][i]:
                df_orders["Order_Delivery_Refunds"][i] = 3       
df_orders.groupby(['Order_Delivery_Refunds'])['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\226306412.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders["Order_Delivery_Refunds"][i] = 3


Order_Delivery_Refunds
0    75560
3       23
Name: order_id, dtype: int64

In [53]:
#Modifying score using the delivery related complaints
order_delivery_complaints = ['Late-Not Received', 'Late-Received', 'Waiting at Store']
df_orders['Order_Delivery_Complaints'] = 0
df_orders.loc[(df_orders['is_complaint'] == 1.0) & (df_orders['comments'].isin(order_delivery_complaints)), 
              'Order_Delivery_Complaints'] = 3
df_orders.groupby(['Order_Delivery_Complaints'])['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\1859044277.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['Order_Delivery_Complaints'] = 0


Order_Delivery_Complaints
0    75360
3      223
Name: order_id, dtype: int64

In [54]:
for i in range(0, len(df_orders)):
    if((df_orders["is_complaint"][i] == 1) and (df_orders["comment_count"][i] > 1)):
        for x in order_delivery_complaints:
            if x in df_orders['comments_split'][i]:
                df_orders["Order_Delivery_Complaints"][i] = 3       
df_orders.groupby(['Order_Delivery_Complaints'])['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2768274370.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders["Order_Delivery_Complaints"][i] = 3


Order_Delivery_Complaints
0    75253
3      330
Name: order_id, dtype: int64

In [55]:
#Final scoring for order delivery based after including complaints and refunds
df_orders['Order_Delivery_Final_inc_C_R'] = df_orders[['Order_Delivery_Final', 'Order_Delivery_Refunds', 'Order_Delivery_Complaints']].max(axis=1)

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\1552185364.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['Order_Delivery_Final_inc_C_R'] = df_orders[['Order_Delivery_Final', 'Order_Delivery_Refunds', 'Order_Delivery_Complaints']].max(axis=1)


<h3>Order Condition Issue</h3>

In [56]:
df_orders[['total_refund_amt', 'refund_reasons', 'comment_count', 'comment_types',
       'comments']]

,total_refund_amt,refund_reasons,comment_count,comment_types,comments
0,NaN,None,NaN,None,None
1,NaN,None,NaN,None,None
2,NaN,None,NaN,None,None
3,NaN,None,NaN,None,None
4,NaN,None,NaN,None,None
...,...,...,...,...,...
75578,NaN,None,1.0,Redemption,System Unavailable
75579,NaN,None,NaN,None,None
75580,NaN,None,NaN,None,None
75581,NaN,None,NaN,None,None


In [57]:
order_condition_complaints = ['Out of Date/Quality','Damaged/Poor Packaging','By Customer']
df_orders['SCORE_Complaints'] = 1
df_orders.loc[(df_orders['is_complaint'] == 1.0) & (df_orders['comments'].isin(order_condition_complaints)), 
              'SCORE_Complaints'] = 3
df_orders.groupby(['SCORE_Complaints'])['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\3301370739.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['SCORE_Complaints'] = 1


SCORE_Complaints
1    75471
3      112
Name: order_id, dtype: int64

In [58]:
for i in range(0, len(df_orders)):
    if((df_orders["is_complaint"][i] == 1) and (df_orders["comment_count"][i] > 1)):
        for x in order_condition_complaints:
            if x in df_orders['comments_split'][i]:
                df_orders["SCORE_Complaints"][i] = 3       
df_orders.groupby(['SCORE_Complaints'])['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\1408131940.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders["SCORE_Complaints"][i] = 3


SCORE_Complaints
1    75403
3      180
Name: order_id, dtype: int64

In [59]:
order_condition_refunds = ['Damaged Item','Expired Item', 'Poor Substitute']
df_orders['SCORE_Refunds'] = 1
df_orders.loc[(df_orders['is_refund'] == 1.0) & (df_orders['refund_reasons'].isin(order_condition_refunds)), 
              'SCORE_Refunds'] = 3
df_orders.groupby(['SCORE_Refunds'])['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\3535286488.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['SCORE_Refunds'] = 1


SCORE_Refunds
1    75583
Name: order_id, dtype: int64

In [60]:
for i in range(0, len(df_orders)):
    if((df_orders["is_refund"][i] == 1) and (df_orders["refund_count"][i] > 1)):
        for x in order_condition_refunds:
            if x in df_orders['refunds_split'][i]:
                df_orders["SCORE_Refunds"][i] = 3       
df_orders.groupby(['SCORE_Refunds'])['order_id'].count()

SCORE_Refunds
1    75583
Name: order_id, dtype: int64

In [61]:
'''
Thresholds used for NPS product quality scoring
9 - 10: 1 : Great Job
7 - 8 : 2 : Average
0 - 6 : 3 : Needs Improvement
'''
df_orders['SCORE_NPS_Product_Quality'] = 0
df_orders.loc[(df_orders["NPS_Product_Quality"] >= 9) &
                      (df_orders["NPS_Product_Quality"] <= 10), "SCORE_NPS_Product_Quality"] = 1
df_orders.loc[(df_orders["NPS_Product_Quality"] >= 7) &
                      (df_orders["NPS_Product_Quality"] <= 8), "SCORE_NPS_Product_Quality"] = 2
df_orders.loc[(df_orders["NPS_Product_Quality"] >= 0) &
                      (df_orders["NPS_Product_Quality"] <= 6), "SCORE_NPS_Product_Quality"] = 3
df_orders.groupby("SCORE_NPS_Product_Quality")['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\1584210032.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['SCORE_NPS_Product_Quality'] = 0


SCORE_NPS_Product_Quality
0    74066
1     1108
2      232
3      177
Name: order_id, dtype: int64

In [62]:
'''
Thresholds used for NPS product freshness scoring
9 - 10: 1 : Great Job
7 - 8 : 2 : Average
0 - 6 : 3 : Needs Improvement
'''
df_orders['SCORE_NPS_Product_Freshness'] = 0
df_orders.loc[(df_orders["NPS_Product_Freshness"] >= 9) &
                      (df_orders["NPS_Product_Freshness"] <= 10), "SCORE_NPS_Product_Freshness"] = 1
df_orders.loc[(df_orders["NPS_Product_Freshness"] >= 7) &
                      (df_orders["NPS_Product_Freshness"] <= 8), "SCORE_NPS_Product_Freshness"] = 2
df_orders.loc[(df_orders["NPS_Product_Freshness"] >= 0) &
                      (df_orders["NPS_Product_Freshness"] <= 6), "SCORE_NPS_Product_Freshness"] = 3
df_orders.groupby("SCORE_NPS_Product_Freshness")['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2902014027.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['SCORE_NPS_Product_Freshness'] = 0


SCORE_NPS_Product_Freshness
0    74068
1     1123
2      225
3      167
Name: order_id, dtype: int64

In [63]:
df_orders['Order_Condition_NPS'] = df_orders[['SCORE_NPS_Product_Quality','SCORE_NPS_Product_Freshness']].max(axis=1)
df_orders.groupby("Order_Condition_NPS")['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\1176823986.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['Order_Condition_NPS'] = df_orders[['SCORE_NPS_Product_Quality','SCORE_NPS_Product_Freshness']].max(axis=1)


Order_Condition_NPS
0    74050
1     1063
2      268
3      202
Name: order_id, dtype: int64

In [64]:
df_orders['Order_Condition_Operational'] = df_orders[['SCORE_Refunds','SCORE_Complaints']].max(axis=1)
df_orders.groupby("Order_Condition_Operational")['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\931343707.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['Order_Condition_Operational'] = df_orders[['SCORE_Refunds','SCORE_Complaints']].max(axis=1)


Order_Condition_Operational
1    75403
3      180
Name: order_id, dtype: int64

In [65]:
df_orders['Order_Condition_Final'] = 0
df_orders.loc[(df_orders['Order_Condition_NPS'] == 1) & 
              (df_orders['Order_Condition_Operational'] == 1), 'Order_Condition_Final'] = 1
df_orders.loc[(df_orders['Order_Condition_NPS'] == 1) & 
              ((df_orders['Order_Condition_Operational'] == 2) | (df_orders['Order_Condition_Operational'] == 3)), 
              'Order_Condition_Final'] = 2
df_orders.loc[(df_orders['Order_Condition_NPS'] == 2) & 
              (df_orders['Order_Condition_Operational'] == 3), 'Order_Condition_Final'] = 3
df_orders.loc[(df_orders['Order_Condition_NPS'] == 2) & 
              ((df_orders['Order_Condition_Operational'] == 1) | (df_orders['Order_Condition_Operational'] == 2)), 
              'Order_Condition_Final'] = 2
df_orders.loc[(df_orders['Order_Condition_NPS'] == 3) & 
              ((df_orders['Order_Condition_Operational'] == 1) | (df_orders['Order_Condition_Operational'] == 2) |
              (df_orders['Order_Condition_Operational'] == 3)), 
              'Order_Condition_Final'] = 3
df_orders.loc[(df_orders['Order_Condition_NPS'] == 0), 'Order_Condition_Final'] = df_orders['Order_Condition_Operational']
df_orders.groupby("Order_Condition_Final")['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\1521342258.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['Order_Condition_Final'] = 0


Order_Condition_Final
1    74938
2      268
3      377
Name: order_id, dtype: int64

<html> <h3> Creating NPS LINK</h3> </html>

In [66]:
#Set up brand id and banner
df_orders['banner_nm'] = ""
df_orders['brand_id'] = ""
df_orders.loc[df_orders['banner'] == 'SAFEWAY', 'banner_nm'] = "safeway"
df_orders.loc[df_orders['banner'] == 'SAFEWAY', 'brand_id'] = "12"
df_orders.loc[df_orders['banner'] == 'UNITED', 'banner_nm'] = "united"
df_orders.loc[df_orders['banner'] == 'UNITED', 'brand_id'] = "17"
df_orders.loc[df_orders['banner'] == 'ACME', 'banner_nm'] = "acme"
df_orders.loc[df_orders['banner'] == 'ACME', 'brand_id'] = "1"
df_orders.loc[df_orders['banner'] == 'ALBERTSONS', 'banner_nm'] = "albertsons"
df_orders.loc[df_orders['banner'] == 'ALBERTSONS', 'brand_id'] = "2"
df_orders.loc[df_orders['banner'] == 'CARRS', 'banner_nm'] = "carrs"
df_orders.loc[df_orders['banner'] == 'CARRS', 'brand_id'] = "5"
df_orders.loc[df_orders['banner'] == 'PAVILIONS', 'banner_nm'] = "pavilions"
df_orders.loc[df_orders['banner'] == 'PAVILIONS', 'brand_id'] = "10"
df_orders.loc[df_orders['banner'] == 'RANDALLS', 'banner_nm'] = "randalls"
df_orders.loc[df_orders['banner'] == 'RANDALLS', 'brand_id'] = "11"
df_orders.loc[df_orders['banner'] == 'STAR', 'banner_nm'] = "starmarket"
df_orders.loc[df_orders['banner'] == 'STAR', 'brand_id'] = "14"
df_orders.loc[df_orders['banner'] == 'TOM THUMB', 'banner_nm'] = "tomthumb"
df_orders.loc[df_orders['banner'] == 'TOM THUMB', 'brand_id'] = "16"
df_orders.loc[df_orders['banner'] == 'VONS', 'banner_nm'] = "vons"
df_orders.loc[df_orders['banner'] == 'VONS', 'brand_id'] = "19"
df_orders.loc[df_orders['banner'] == 'JEWEL-OSCO', 'banner_nm'] = "jewelosco"
df_orders.loc[df_orders['banner'] == 'JEWEL-OSCO', 'brand_id'] = "7"
df_orders.loc[df_orders['banner'] == 'SHAW\'S', 'banner_nm'] = "shaws"
df_orders.loc[df_orders['banner'] == 'SHAW\'S', 'brand_id'] = "13"
df_orders.loc[df_orders['banner'] == 'ANDRONICO\'S', 'banner_nm'] = "andronicos"
df_orders.loc[df_orders['banner'] == 'ANDRONICO\'S', 'brand_id'] = "25"
df_orders.loc[df_orders['banner'] == 'BALDUCCI\'S', 'banner_nm'] = "balduccis"
df_orders.loc[df_orders['banner'] == 'BALDUCCI\'S', 'brand_id'] = "20"
df_orders.loc[df_orders['banner'] == 'HAGGEN', 'banner_nm'] = "haggen"
df_orders.loc[df_orders['banner'] == 'HAGGEN', 'brand_id'] = "6"
df_orders.loc[df_orders['banner'] == 'KINGS', 'banner_nm'] = "kings"
df_orders.loc[df_orders['banner'] == 'KINGS', 'brand_id'] = "22"
df_orders.groupby(['banner', 'banner_nm', 'brand_id'])['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\210176783.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['banner_nm'] = ""
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\210176783.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['brand_id'] = ""


banner                banner_nm   brand_id
ACME                  acme        1            3447
ALBERTSONS            albertsons  2           12766
ANDRONICO'S           andronicos  25             32
BALDUCCI'S            balduccis   20             17
CARRS                 carrs       5             318
HAGGEN                haggen      6             189
JEWEL-OSCO            jewelosco   7            9324
KINGS                 kings       22             56
MARKET STREET                                    74
PAK N SAV BY SAFEWAY                            357
PAVILIONS             pavilions   10            907
RANDALLS              randalls    11            546
SAFEWAY               safeway     12          32042
SHAW'S                shaws       13           6234
STAR                  starmarket  14            818
TOM THUMB             tomthumb    16           2178
VONS                  vons        19           6207
Name: order_id, dtype: int64

In [67]:
#set up subscription flag
df_orders['subscription_flag'] = "2"
df_orders.loc[df_orders['fresh_pass_hhs'] == True, 'subscription_flag'] = "1"
df_orders.groupby(['subscription_flag'])['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2785151339.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['subscription_flag'] = "2"


subscription_flag
1    11868
2    63715
Name: order_id, dtype: int64

In [68]:
df_orders['survey_taken'] = ""
df_orders.loc[df_orders['survey_id'].isna() == False, 'survey_taken'] = "Yes"
df_orders.loc[df_orders['survey_id'].isna() == True, 'survey_taken'] = "No"
df_orders.groupby(['survey_taken'])['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\456802123.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['survey_taken'] = ""


survey_taken
No     73999
Yes     1584
Name: order_id, dtype: int64

In [69]:
#Setting service level
df_orders['service_type'] = df_orders['svclvl_type']
df_orders.loc[df_orders['svclvl_type'] == "STANDARD", 'service_type'] = "REGULAR"
df_orders.groupby(['svclvl_type', 'service_type'])['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\1623664451.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['service_type'] = df_orders['svclvl_type']


svclvl_type  service_type
EXPRESS      EXPRESS         32102
FLASH        FLASH            3725
STANDARD     REGULAR         39756
Name: order_id, dtype: int64

In [70]:
df_orders['carrier'].unique()

array([None, 'UBR', 'DDS', 'Fleet', 'Not_Tracked'], dtype=object)

In [71]:
#Setting carrier
df_orders['carrier_name'] = df_orders['carrier']
df_orders.loc[df_orders['carrier'] == "Not_Tracked", 'carrier_name'] = "RP"
df_orders.loc[df_orders['carrier'].isna() == True, 'carrier_name'] = "RP"
df_orders.groupby(['carrier_name'])['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\213658029.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['carrier_name'] = df_orders['carrier']


carrier_name
DDS      15975
Fleet     2170
RP       51144
UBR       6294
Name: order_id, dtype: int64

In [72]:
#creating NPS link for each order
df_orders['NPS link'] = ""
for i in range(0, len(df_orders)):
    order_num = str(df_orders["order_id"][i])
    order_tm = df_orders["txn_tm"][i].strftime("%m%d%Y%H%M%S")
    banner = df_orders["banner_nm"][i]
    brand = df_orders["brand_id"][i]
    store = df_orders["store_id"][i]
    delivery_type = df_orders["delivery_type"][i]
    carrier = df_orders["carrier_name"][i]
    subscription = df_orders["subscription_flag"][i]
    service_level = df_orders["service_type"][i]
    if(df_orders['survey_taken'][i] == "No"):
        if(delivery_type == "DUG"):
            df_orders['NPS link'][i] = "https://survey3.medallia.com/?dug1-"+banner+"&brand="+brand+"&fulfillmentorderid="+order_num+"&date="+order_tm+"&isSubscription="+subscription+"&SL="+service_level+"&isMP=2&SC=1"
        elif(delivery_type == "DELIVERY"):
            #df_orders['NPS link'][i] = "https://survey3.medallia.com/?deliv-"+banner+"&brand="+brand+"&fulfillmentorderid="+order_num+"&date="+order_tm+"&type="+carrier+"&isSubscription="+subscription+"&SL="+service_level+"&isMP=2&SC=1"
            df_orders['NPS link'][i] = "https://survey3.medallia.com/?deliv-{}&brand={}&store={}&fulfillmentorderid={}&date={}&type={}&isSubscription={}&SL={}&isMP=2&SC=1".format(banner, brand, store, order_num, order_tm, carrier, subscription, service_level)

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\4090464214.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['NPS link'] = ""
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\4090464214.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders['NPS link'][i] = "https://survey3.medallia.com/?dug1-"+banner+"&brand="+brand+"&fulfillmentorderid="+order_num+"&date="+order_tm+"&isSubscription="+subscription+"&SL="+service_level+"&isMP=2&SC=1"
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\4090464214.py:18: SettingWithCopyWarning: 
A value i

<html> <h1> Households Data </h1> </html>

In [73]:
df_hhs= df_hhs.rename(columns=str.lower)
df_orders["household_id"] = df_orders["household_id"].astype('Int64')
df_hhs["household_id"] = df_hhs["household_id"].astype('Int64')

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\3304416813.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders["household_id"] = df_orders["household_id"].astype('Int64')


In [74]:
#Left join transactions qualified with the households data
df_orders_hhs = pd.merge(df_orders, df_hhs, left_on="household_id", right_on="household_id", how='left', suffixes=('','hhs'))

In [75]:
#Keeping the status of the customer prior to the transaction date
df_orders_hhs = df_orders_hhs[(df_orders_hhs["txn_dte"] > df_orders_hhs["latest_ecom_txn_dte"]) | 
             df_orders_hhs["latest_ecom_txn_dte"].isna()]

In [76]:
#Setting the customer segment column based on the flag
df_orders_hhs.loc[df_orders_hhs["bnc_to_aci_flag"] == 1, "Customer_Segment"] = "BNC to ACI"
df_orders_hhs.loc[(df_orders_hhs["bnc_to_aci_flag"] == 0) & (df_orders_hhs["bnc_to_ecom_flag"] == 1), "Customer_Segment"] = "BNC to eComm"
df_orders_hhs.loc[df_orders_hhs["acquired_prior_6month"] == 1, "Customer_Segment"] = "Acquired prior to 6 months"
df_orders_hhs.loc[df_orders_hhs["acquired_within_6month"] == 1, "Customer_Segment"] = "Acquired within 6 months"
df_orders_hhs.groupby("Customer_Segment")["household_id"].count()

Customer_Segment
Acquired prior to 6 months    54828
Acquired within 6 months      11415
BNC to ACI                     1228
BNC to eComm                   3258
Name: household_id, dtype: int64

In [77]:
#Verifying the customer segment set
df_orders_hhs[(df_orders_hhs["bnc_to_aci_flag"] == 0) &
             (df_orders_hhs["bnc_to_ecom_flag"] == 0) &
             (df_orders_hhs["acquired_prior_6month"] == 0) &
             (df_orders_hhs["acquired_within_6month"] == 1)]["household_id"].count()

11415

In [78]:
#Bucketizing the number of transations 
df_orders_hhs.loc[(df_orders_hhs["count_ecom_txn_last6month"] == 0) |
               (df_orders_hhs["count_ecom_txn_last6month"].isnull()), "Ecomm txns in last 6 months"] = "0 txns"
df_orders_hhs.loc[(df_orders_hhs["count_ecom_txn_last6month"] > 0) &
               (df_orders_hhs["count_ecom_txn_last6month"] <= 4), "Ecomm txns in last 6 months"] = "1-4 txns"
df_orders_hhs.loc[df_orders_hhs["count_ecom_txn_last6month"] > 4, "Ecomm txns in last 6 months"] = "5+ txns"
df_orders_hhs.groupby("Ecomm txns in last 6 months")["household_id"].count()

Ecomm txns in last 6 months
0 txns       6853
1-4 txns    12915
5+ txns     51246
Name: household_id, dtype: int64

In [79]:
#Getting the minimum transaction date in the transaction file
min_txn_date = df_created_orders["txn_dte"].min()
df_orders_hhs["latest_ecom_txn_dte"] = df_orders_hhs["latest_ecom_txn_dte"].astype(str)

In [80]:
df_orders_hhs.reset_index(inplace=True)

In [81]:
#Setting the NaT values to -1000.0 for comparison
df_orders_hhs.loc[df_orders_hhs["latest_ecom_txn_dte"] == "NaT", "Time since last ecomm txn"] = -1000.0
df_orders_hhs.loc[df_orders_hhs["latest_ecom_txn_dte"] == "None", "Time since last ecomm txn"] = -1000.0
df_orders_hhs.loc[df_orders_hhs["latest_ecom_txn_dte"] == "nan", "Time since last ecomm txn"] = -1000.0

In [82]:
#Getting the time since last ecomm transaction
from datetime import datetime
for i in range(0, len(df_orders_hhs)):
    if df_orders_hhs["Time since last ecomm txn"][i] != -1000.0:
        df_orders_hhs["Time since last ecomm txn"][i] = (((min_txn_date - (datetime.strptime(df_orders_hhs["latest_ecom_txn_dte"][i], '%Y-%m-%d')).date()).days)/7)
        #df_orders_hhs["Time since last ecomm txn"][i] = (((min_txn_date - df_orders_hhs["latest_ecom_txn_dte"][i]).days)/7)
#df_hhs_Nov.groupby("Time since last ecomm txn")["HOUSEHOLD_ID"].count()
#df_orders_hhs["Time since last ecomm txn"][4763]

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2409533487.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders_hhs["Time since last ecomm txn"][i] = (((min_txn_date - (datetime.strptime(df_orders_hhs["latest_ecom_txn_dte"][i], '%Y-%m-%d')).date()).days)/7)


In [83]:
#Bucketising the time since last transaction into weeks timeframe
df_orders_hhs.loc[df_orders_hhs["Time since last ecomm txn"] == -1000.0, "Weeks since last ecomm txn"] = "No ecomm txn"
df_orders_hhs.loc[(df_orders_hhs["Time since last ecomm txn"] >= 0.0) &
               (df_orders_hhs["Time since last ecomm txn"] < 4.0), "Weeks since last ecomm txn"] = "<4 weeks"
df_orders_hhs.loc[(df_orders_hhs["Time since last ecomm txn"] >= 4.0) &
               (df_orders_hhs["Time since last ecomm txn"] < 12.0), "Weeks since last ecomm txn"] = "4-12 weeks"
df_orders_hhs.loc[df_orders_hhs["Time since last ecomm txn"] >= 12.0, "Weeks since last ecomm txn"] = "12+ weeks"
df_orders_hhs.groupby("Weeks since last ecomm txn")["household_id"].count()

Weeks since last ecomm txn
12+ weeks        4074
4-12 weeks       7868
<4 weeks        54301
No ecomm txn     4771
Name: household_id, dtype: int64

In [84]:
df_orders_hhs = df_orders_hhs.loc[df_orders_hhs["Customer_Segment"].isna() == False]

In [85]:
df_orders_hhs[(df_orders_hhs["Customer_Segment"] == "Acquired within 6 months") &
             (df_orders_hhs["Ecomm txns in last 6 months"] == "0 txns") &
             (df_orders_hhs["Weeks since last ecomm txn"] == "12+ weeks")]
df_orders_hhs.loc[df_orders_hhs["urbanicity"].isna() == True, "urbanicity"] = "Unknown"

In [86]:
'''
  !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Possibly in this section add in the logic to split the orders by househholds that have already repeated.
 I want to see the customers original TXN Test Experience so that Control can be given a Control Experience again,
  while customers that receieved an offer of Apology before only get the score. The idea behind this is that we 
  will now be able to track if the customers retention has improved through Proactive Communication or worsened. 
  !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
'''

'\n  !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\nPossibly in this section add in the logic to split the orders by househholds that have already repeated.\n I want to see the customers original TXN Test Experience so that Control can be given a Control Experience again,\n  while customers that receieved an offer of Apology before only get the score. The idea behind this is that we \n  will now be able to track if the customers retention has improved through Proactive Communication or worsened. \n  !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n'

In [87]:
#Setting custom segments for certain exclusions
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Suburban") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "0 txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "12+ weeks") , "Segment Number"] = 1
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Urban") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "0 txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "12+ weeks") , "Segment Number"] = 2
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Rural") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "0 txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "12+ weeks") , "Segment Number"] = 3
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Unknown") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "0 txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "12+ weeks") , "Segment Number"] = 4
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Suburban") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "0 txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "No ecomm txn") , "Segment Number"] = 5
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Urban") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "0 txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "No ecomm txn") , "Segment Number"] = 6
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Rural") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "0 txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "No ecomm txn") , "Segment Number"] = 7
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Unknown") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "0 txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "No ecomm txn") , "Segment Number"] = 8
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Suburban") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "1-4 txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "12+ weeks") , "Segment Number"] = 9
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Urban") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "1-4 txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "12+ weeks") , "Segment Number"] = 10
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Rural") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "1-4 txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "12+ weeks") , "Segment Number"] = 11
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Unknown") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "1-4 txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "12+ weeks") , "Segment Number"] = 12
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Suburban") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "1-4 txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "4-12 weeks") , "Segment Number"] = 13
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Urban") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "1-4 txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "4-12 weeks") , "Segment Number"] = 14
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Rural") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "1-4 txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "4-12 weeks") , "Segment Number"] = 15
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Unknown") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "1-4 txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "4-12 weeks") , "Segment Number"] = 16
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Suburban") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "1-4 txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "<4 weeks") , "Segment Number"] = 17
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Urban") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "1-4 txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "<4 weeks") , "Segment Number"] = 18
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Rural") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "1-4 txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "<4 weeks") , "Segment Number"] = 19
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Unknown") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "1-4 txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "<4 weeks") , "Segment Number"] = 20
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Suburban") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "5+ txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "12+ weeks") , "Segment Number"] = 21
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Urban") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "5+ txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "12+ weeks") , "Segment Number"] = 22
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Rural") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "5+ txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "12+ weeks") , "Segment Number"] = 23
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Unknown") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "5+ txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "12+ weeks") , "Segment Number"] = 24
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Suburban") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "5+ txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "4-12 weeks") , "Segment Number"] = 25
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Urban") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "5+ txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "4-12 weeks") , "Segment Number"] = 26
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Rural") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "5+ txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "4-12 weeks") , "Segment Number"] = 27
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Unknown") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "5+ txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "4-12 weeks") , "Segment Number"] = 28
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Suburban") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "5+ txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "<4 weeks") , "Segment Number"] = 29
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Urban") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "5+ txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "<4 weeks") , "Segment Number"] = 30
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Rural") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "5+ txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "<4 weeks") , "Segment Number"] = 31
df_orders_hhs.loc[(df_orders_hhs["urbanicity"] == "Unknown") &
                  (df_orders_hhs["Ecomm txns in last 6 months"] == "5+ txns") &
                  (df_orders_hhs["Weeks since last ecomm txn"] == "<4 weeks") , "Segment Number"] = 32
#df_orders_hhs.loc[(df_orders_hhs["urbanicity"].isna()) , "Segment Number"] = 25

In [88]:
df_orders_hhs.loc[df_orders_hhs["txn_tm"] == "NaT"]
#df_orders_hhs = df_orders_hhs.drop(columns=["level_0", "index"])
df_orders_hhs = df_orders_hhs.drop(columns=["index"])
df_orders_hhs.reset_index(inplace=True)
df_orders_hhs.count()

index                          70729
order_status_nm                70729
version_id                     70729
order_id                       70729
order_dt                       70729
                               ...  
Customer_Segment               70729
Ecomm txns in last 6 months    70729
Time since last ecomm txn      70729
Weeks since last ecomm txn     70729
Segment Number                 70729
Length: 152, dtype: int64

In [89]:
df_orders_hhs_v2 = df_orders_hhs
df_orders_hhs_v2[['household_id', 'Ecomm txns in last 6 months', 'Weeks since last ecomm txn', 'urbanicity', 'order_id', 'banner', 'store_id']]

,household_id,Ecomm txns in last 6 months,Weeks since last ecomm txn,urbanicity,order_id,banner,store_id
0,961054028361,5+ txns,<4 weeks,Rural,83972697,JEWEL-OSCO,3515
1,990006310858,5+ txns,<4 weeks,Suburban,83979669,SAFEWAY,1765
2,990032319901,5+ txns,<4 weeks,Suburban,83956317,ALBERTSONS,3333
3,990033806419,5+ txns,<4 weeks,Rural,83973889,ACME,3961
4,903041795601,5+ txns,<4 weeks,Rural,84008498,ALBERTSONS,130
...,...,...,...,...,...,...,...
70724,729011957623,1-4 txns,4-12 weeks,Suburban,83987653,SAFEWAY,2713
70725,729012061219,5+ txns,<4 weeks,Suburban,83925305,SAFEWAY,1078
70726,990073993867,1-4 txns,<4 weeks,Suburban,84011923,SAFEWAY,1215
70727,990044204431,5+ txns,<4 weeks,Rural,83997261,ALBERTSONS,2798


In [90]:
#Removing orders where data is not available for operational metrics, NPS, complaints and refunds
df_orders_hhs_v2 = df_orders_hhs_v2[((df_orders_hhs_v2['Order_Accuracy_Final_inc_C_R'] != 0) &
                     (df_orders_hhs_v2['Order_Delivery_Final_inc_C_R'] != 0) &
                     (df_orders_hhs_v2['Order_Condition_Final'] != 0))]

In [91]:
#Assigning overall order experience 
df_orders_hhs_v2['Overall_order_experience'] = "Good"
df_orders_hhs_v2.loc[((df_orders_hhs_v2['Order_Accuracy_Final_inc_C_R'] == 3) |
                     (df_orders_hhs_v2['Order_Delivery_Final_inc_C_R'] == 3) |
                     (df_orders_hhs_v2['Order_Condition_Final'] == 3)), 'Overall_order_experience'] = "Bad"
df_orders_hhs_v2.groupby(['Overall_order_experience'])['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\1101499249.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders_hhs_v2['Overall_order_experience'] = "Good"


Overall_order_experience
Bad     16584
Good    53920
Name: order_id, dtype: int64

In [92]:
#assigning theme based on overall order experience
df_orders_hhs_v2['Theme'] = ""
df_orders_hhs_v2.loc[df_orders_hhs_v2['Overall_order_experience'] == 'Bad', 'Theme'] = 'RetentionV2Bad'
df_orders_hhs_v2.loc[df_orders_hhs_v2['Overall_order_experience'] == 'Good', 'Theme'] = 'RetentionV2Good'
df_orders_hhs_v2.groupby('Theme')['order_id'].count()

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\1857010594.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders_hhs_v2['Theme'] = ""


Theme
RetentionV2Bad     16584
RetentionV2Good    53920
Name: order_id, dtype: int64

In [93]:
df_orders_hhs_v2['Product Availability'] = 'Order Accuracy'
df_orders_hhs_v2['Order Delivery'] = 'Order Timeliness'
df_orders_hhs_v2['Order Condition'] = 'Product Condition'

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2910937957.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders_hhs_v2['Product Availability'] = 'Order Accuracy'
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2910937957.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders_hhs_v2['Order Delivery'] = 'Order Timeliness'
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2910937957.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

In [94]:
#Assigning score and alt text for the email template
#Updated with New Links for GREAT
#OLD LINKS DO NOT USE "https://images.albertsons-media.com/is/image/ABS/sc-great-job-1-C10?$emailcampaign-badge$"
df_orders_hhs_v2['Product Availability Alt Text'] = ""
df_orders_hhs_v2['Product Availability Score'] = ""

df_orders_hhs_v2.loc[df_orders_hhs_v2['Order_Accuracy_Final_inc_C_R'] == 3, 'Product Availability Alt Text'] = "Order Accuracy needed improvement"
df_orders_hhs_v2.loc[df_orders_hhs_v2['Order_Accuracy_Final_inc_C_R'] == 3, 'Product Availability Score'] = "https://images.albertsons-media.com/is/image/ABS/sc-needs-improvement-1-C10?$emailcampaign-badge$"

df_orders_hhs_v2.loc[df_orders_hhs_v2['Order_Accuracy_Final_inc_C_R'] == 2, 'Product Availability Alt Text'] = "Order Accuracy was average"
df_orders_hhs_v2.loc[df_orders_hhs_v2['Order_Accuracy_Final_inc_C_R'] == 2, 'Product Availability Score'] = "https://images.albertsons-media.com/is/image/ABS/sc-average-1-C10?$emailcampaign-badge$"

df_orders_hhs_v2.loc[df_orders_hhs_v2['Order_Accuracy_Final_inc_C_R'] == 1, 'Product Availability Alt Text'] = "Order Accuracy was great"
df_orders_hhs_v2.loc[df_orders_hhs_v2['Order_Accuracy_Final_inc_C_R'] == 1, 'Product Availability Score'] = "https://images.albertsons-media.com/is/image/ABS/sc-great-1-C10?$emailcampaign-badge$"

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\134998012.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders_hhs_v2['Product Availability Alt Text'] = ""
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\134998012.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders_hhs_v2['Product Availability Score'] = ""


In [95]:
#Assigning score and alt text for the email template
#Updated with New Links for GREAT
#OLD LINKS DO NOT USE "https://images.albertsons-media.com/is/image/ABS/sc-great-job-1-C10?$emailcampaign-badge$"
df_orders_hhs_v2['Order Delivery Alt Text'] = ""
df_orders_hhs_v2['Order Delivery Score'] = ""

df_orders_hhs_v2.loc[df_orders_hhs_v2['Order_Delivery_Final_inc_C_R'] == 3, 'Order Delivery Alt Text'] = "Order Timeliness needed improvement"
df_orders_hhs_v2.loc[df_orders_hhs_v2['Order_Delivery_Final_inc_C_R'] == 3, 'Order Delivery Score'] = "https://images.albertsons-media.com/is/image/ABS/sc-needs-improvement-1-C10?$emailcampaign-badge$"

df_orders_hhs_v2.loc[df_orders_hhs_v2['Order_Delivery_Final_inc_C_R'] == 2, 'Order Delivery Alt Text'] = "Order Timeliness was average"
df_orders_hhs_v2.loc[df_orders_hhs_v2['Order_Delivery_Final_inc_C_R'] == 2, 'Order Delivery Score'] = "https://images.albertsons-media.com/is/image/ABS/sc-average-2-C10?$emailcampaign-badge$"

df_orders_hhs_v2.loc[df_orders_hhs_v2['Order_Delivery_Final_inc_C_R'] == 1, 'Order Delivery Alt Text'] = "Order Timeliness was great"
df_orders_hhs_v2.loc[df_orders_hhs_v2['Order_Delivery_Final_inc_C_R'] == 1, 'Order Delivery Score'] = "https://images.albertsons-media.com/is/image/ABS/sc-great-2-C10?$emailcampaign-badge$"

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\394356995.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders_hhs_v2['Order Delivery Alt Text'] = ""
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\394356995.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders_hhs_v2['Order Delivery Score'] = ""


In [96]:
#Assigning score and alt text for the email template
#Updated with New Links for GREAT
#OLD LINKS DO NOT USE "https://images.albertsons-media.com/is/image/ABS/sc-great-job-1-C10?$emailcampaign-badge$"
df_orders_hhs_v2['Order Condition Alt Text'] = ""
df_orders_hhs_v2['Order Condition Score'] = ""

df_orders_hhs_v2.loc[df_orders_hhs_v2['Order_Condition_Final'] == 3, 'Order Condition Alt Text'] = "Product condition needed improvement"
df_orders_hhs_v2.loc[df_orders_hhs_v2['Order_Condition_Final'] == 3, 'Order Condition Score'] = "https://images.albertsons-media.com/is/image/ABS/sc-needs-improvement-3-C10?$emailcampaign-badge$"

df_orders_hhs_v2.loc[df_orders_hhs_v2['Order_Condition_Final'] == 2, 'Order Condition Alt Text'] = "Product condition was average"
df_orders_hhs_v2.loc[df_orders_hhs_v2['Order_Condition_Final'] == 2, 'Order Condition Score'] = "https://images.albertsons-media.com/is/image/ABS/sc-average-3-C10?$emailcampaign-badge$"

df_orders_hhs_v2.loc[df_orders_hhs_v2['Order_Condition_Final'] == 1, 'Order Condition Alt Text'] = "Product condition was great"
df_orders_hhs_v2.loc[df_orders_hhs_v2['Order_Condition_Final'] == 1, 'Order Condition Score'] = "https://images.albertsons-media.com/is/image/ABS/sc-great-3-C10?$emailcampaign-badge$"

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\698740562.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders_hhs_v2['Order Condition Alt Text'] = ""
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\698740562.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders_hhs_v2['Order Condition Score'] = ""


In [97]:
##Assigning dynamic text for the email template
df_orders_hhs_v2['Order Summary'] = ""
df_orders_hhs_v2.loc[(df_orders_hhs_v2['survey_taken'] == "No") & 
                    (df_orders_hhs_v2['Overall_order_experience'] == "Bad"), 'Order Summary'] = "Here's how we internally rated your order based on our high expectations of customer service.*"
df_orders_hhs_v2.loc[(df_orders_hhs_v2['survey_taken'] == "Yes") & 
                    (df_orders_hhs_v2['Overall_order_experience'] == "Bad"), 'Order Summary'] = "Rest assured we're making changes at our end based on your helpful feedback. Here's a summary of your order.*"
df_orders_hhs_v2.loc[(df_orders_hhs_v2['survey_taken'] == "No") & 
                    (df_orders_hhs_v2['Overall_order_experience'] == "Good"), 'Order Summary'] = "Here's how we internally rated your order based on our high expectations of customer service.*"
df_orders_hhs_v2.loc[(df_orders_hhs_v2['survey_taken'] == "Yes") & 
                    (df_orders_hhs_v2['Overall_order_experience'] == "Good"), 'Order Summary'] = "Your recent feedback will be used to help us improve our services. Here's a summary of your order.*"

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\1043877141.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders_hhs_v2['Order Summary'] = ""


In [98]:
Order_Experience = df_orders_hhs_v2['Overall_order_experience'].unique()
Order_Experience
Segments = df_orders_hhs_v2["Segment Number"].unique()

In [99]:
segmentation

,segment_number,ecomm_txns_in_last_6_months,weeks_since_last_ecomm_txn,urbanicity,10_OFF,5_OFF,apology,control
0,1,0 txns,12+ weeks,Suburban,0.25,0.25,0.5,0.0
1,32,5+ txns,<4 weeks,Unknown,0.00,0.10,0.9,0.0
2,31,5+ txns,<4 weeks,Rural,0.00,0.10,0.9,0.0
3,30,5+ txns,<4 weeks,Urban,0.00,0.10,0.9,0.0
4,29,5+ txns,<4 weeks,Suburban,0.00,0.10,0.9,0.0
5,28,5+ txns,4-12 weeks,Unknown,0.25,0.25,0.5,0.0
6,27,5+ txns,4-12 weeks,Rural,0.25,0.25,0.5,0.0
7,26,5+ txns,4-12 weeks,Urban,0.25,0.25,0.5,0.0
8,25,5+ txns,4-12 weeks,Suburban,0.25,0.25,0.5,0.0
9,24,5+ txns,12+ weeks,Unknown,0.25,0.25,0.5,0.0


In [100]:
#Randomly assigning customers to different a test group after separating them by bad experience qualification
control = pd.DataFrame()
test_no_offer = pd.DataFrame()
test_10dollar = pd.DataFrame()
test_5dollar = pd.DataFrame()
for f in Order_Experience:
    for s in Segments:
        #print(s)
        #df1 = pd.DataFrame()
        df1 = df_orders_hhs_v2[(df_orders_hhs_v2["Segment Number"] == s) & (df_orders_hhs_v2["Overall_order_experience"] == f)]
        #print(segmentation[segmentation["Segment Number"] == s]["Control"])
        for i in range(0,len(segmentation)):
            if(segmentation["segment_number"][i] == s):
                c_frac = segmentation["control"][i]
        #print(c_frac)
        c1 = df1.sample(frac=c_frac)
        #print(s,df1["HOUSEHOLD_ID"].count(),c1["HOUSEHOLD_ID"].count())
        control = control.append(c1, ignore_index=True)
        #del df1
        df1 = df1.drop(c1.index)
        #df1.reset_index(inplace=True)
        for i in range(0,len(segmentation)):
            if(segmentation["segment_number"][i] == s):
                t10_frac = segmentation["10_OFF"][i]
        #print(t10_frac)
        final_t10 = t10_frac/(1-c_frac)
        #print(final_t10)
        t1 = df1.sample(frac=final_t10)
        test_10dollar = test_10dollar.append(t1, ignore_index=True)
        #test_10dollar.append(df1.sample(frac=0.33))
        df1 = df1.drop(t1.index)
        for i in range(0,len(segmentation)):
            if(segmentation["segment_number"][i] == s):
                t5_frac = segmentation["5_OFF"][i]
        #print(t5_frac)
        final_t5 = t5_frac/(1-(c_frac+t10_frac))
        #print(final_t5)
        t2 = df1.sample(frac=final_t5)
        test_5dollar = test_5dollar.append(t2, ignore_index=True)
        ##test_5dollar.append(df1.sample(frac=0.5))
        df1 = df1.drop(t2.index)
        test_no_offer = test_no_offer.append(df1)
        

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\3721607876.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  control = control.append(c1, ignore_index=True)
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\3721607876.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_10dollar = test_10dollar.append(t1, ignore_index=True)
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\3721607876.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_5dollar = test_5dollar.append(t2, ignore_index=True)
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\3721607876.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_no_offer 

In [101]:
df_orders_hhs_v2.groupby(["Segment Number", "Ecomm txns in last 6 months", "Weeks since last ecomm txn", "urbanicity"])["household_id"].count()

Segment Number  Ecomm txns in last 6 months  Weeks since last ecomm txn  urbanicity
1.0             0 txns                       12+ weeks                   Suburban       1040
2.0             0 txns                       12+ weeks                   Urban           508
3.0             0 txns                       12+ weeks                   Rural           443
4.0             0 txns                       12+ weeks                   Unknown          86
5.0             0 txns                       No ecomm txn                Suburban       2346
6.0             0 txns                       No ecomm txn                Urban          1111
7.0             0 txns                       No ecomm txn                Rural           882
8.0             0 txns                       No ecomm txn                Unknown         131
9.0             1-4 txns                     12+ weeks                   Suburban        891
10.0            1-4 txns                     12+ weeks                   Urban 

In [102]:
#Setting the test experience column based on the test group
test_10dollar["Test Experience"] = "$10 off"
test_5dollar["Test Experience"] = "$5 off"
control["Test Experience"] = "Control"
test_no_offer["Test Experience"] = "No offer"

In [103]:
df_final = pd.concat([test_10dollar,test_5dollar,test_no_offer])
df_test = pd.concat([test_10dollar,test_5dollar,test_no_offer, control])

In [104]:
# Updated: 4/10/23 (Nick)
# This will defult the test households below to Apology Only Emails
# '990072161305', '990059087520', '990075616372', '932046858249', '990053617034'
TestHHS_IDs = [990072161305, 990059087520, 990075616372, 932046858249, 990053617034]
df_test.loc[df_test['household_id'].isin(TestHHS_IDs), 'Test Experience'] = 'No offer'

In [105]:
#Updated: 4/21/23 (Nick)
#This will take the customer who have placed orders are specific holdout stores and replace their Test Experience with Control. 
HoldoutSTORES = [58,68,80,116,122,123,224,754,1074,1118,1190,1296,1403,2265,2444,2602,3003,
3073,3084,3088,3191,3220,3223,3224,3262,3278,3284,3290,3341,3346,3349,3422,3425,3429,3446,
3461,3473,3476,3477,3481,3484,3491,3495,3730,3741,3770,3775,4013,4043,4057,4228]
df_test.loc[df_test['store_id'].isin(HoldoutSTORES), 'Test Experience'] = 'Control'


In [106]:
############################################################################################
############################################################################################
## Data to be imported for Testing For Faezeh 
## DATASCIENCE SECTION
############################################################################################
############################################################################################
df_TESTING_DS = pd.concat([test_10dollar,test_5dollar,test_no_offer, control])
df_TESTING_DS = df_TESTING_DS[["household_id", "order_id", "order_dt", "txn_id", "Customer_Segment", "Ecomm txns in last 6 months", "Weeks since last ecomm txn", "txn_dte", "banner", "store_id", "Fulfillment Experience", "Repeat_Customer", "urbanicity", "Test Experience"]]
df_TESTING_DS.columns = df_TESTING_DS.columns.str.upper()
df_TESTING_DS['EMAIL_DATE'] = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
df_TESTING_DS.rename(columns = {'ECOMM TXNS IN LAST 6 MONTHS':'ECOMM_TXNS_IN_LAST_6_MONTHS', 'WEEKS SINCE LAST ECOMM TXN': 'WEEKS_SINCE_LAST_ECOMM_TXN', 'TEST EXPERIENCE':'TEST_EXPERIENCE', 'FULFILLMENT EXPERIENCE':'FULFILLMENT_EXPERIENCE'}, inplace = True)

df_ds_export = df_test[['household_id', 'order_id', 'txn_dte', 'delivery_type', 'banner', 'store_id','Customer_Segment','Ecomm txns in last 6 months','Weeks since last ecomm txn','urbanicity', 'Test Experience']]

#STEP#1: Would be to remove the existing data from the table first 
#df_TESTING_DS_UPLOAD = spark.createDataFrame(df_TESTING_DS)
#sfUtils = sc._jvm.net.snowflake.spark.snowflake.Utils
#sfUtils.runQuery(db_options,"DELETE FROM EDM_BIZOPS_PRD.FBI.ESA_MIR_DS_HOUSEHOLD_EXPORT")
#Step#2: Would be to add the data back into the table.
#df_TESTING_DS_UPLOAD.write.format("net.snowflake.spark.snowflake").options(**db_options).option("dbtable", "ESA_MIR_DS_HOUSEHOLD_EXPORT").mode('append').save()
#Loading final output file to EDM
#conn = SFConnection()
#conn.add_name("nacke08")
#conn.test_conn()
#conn.load_dataframe_REPLACE(df_ds_export, 'ESA_PROACTIVE_DS_HOUSEHOLD_EXPORT')
#conn.close_conn()

############################################################################################
############################################################################################

KeyError: "['Fulfillment Experience', 'Repeat_Customer'] not in index"

In [107]:
#Updated: 6/20/23 (Nick)
#This will look for customers that have already received a coupon in the past 30 days and switch their Test Experience to 'No offer'
#This is completed by comparing against a list of existing households who have received coupon in the past 30 days. This query is referenced at the top of the file. 
df_test.loc[(df_test['household_id'].isin(RepeatCustomers['hhs_id'])) & (df_test['Test Experience'] == '$10 off'), 'Test Experience'] = 'No offer'
df_test.loc[(df_test['household_id'].isin(RepeatCustomers['hhs_id'])) & (df_test['Test Experience'] == '$5 off'), 'Test Experience'] = 'No offer'

In [108]:
#Assigning dynamic text relating to business value for the email template
df_test['Business value'] = ""
df_test.loc[(df_test['Test Experience'] == '$10 off') &
           (df_test['Overall_order_experience'] == "Bad"), 'Business value'] = "We know we can do better, and hope you'll accept this offer for your next online order."
df_test.loc[(df_test['Test Experience'] == '$5 off') &
           (df_test['Overall_order_experience'] == "Bad"), 'Business value'] = "We know we can do better, and hope you'll accept this offer for your next online order."
df_test.loc[(df_test['Test Experience'] == 'No offer') &
           (df_test['Overall_order_experience'] == "Bad"), 'Business value'] = "We value your business, and know we can do better. Look forward to serving you again soon."
df_test.loc[(df_test['Test Experience'] == 'Control') &
           (df_test['Overall_order_experience'] == "Bad"), 'Business value'] = "We value your business, and know we can do better. Look forward to serving you again soon."
df_test.loc[(df_test['Test Experience'] == '$10 off') &
           (df_test['Overall_order_experience'] == "Good"), 'Business value'] = "As our way of saying thanks, redeem this offer to save on your next online order."
df_test.loc[(df_test['Test Experience'] == '$5 off') &
           (df_test['Overall_order_experience'] == "Good"), 'Business value'] = "As our way of saying thanks, redeem this offer to save on your next online order."
df_test.loc[(df_test['Test Experience'] == 'No offer') &
           (df_test['Overall_order_experience'] == "Good"), 'Business value'] = "We truly value your business and look forward to serving you again soon!"
df_test.loc[(df_test['Test Experience'] == 'Control') &
           (df_test['Overall_order_experience'] == "Good"), 'Business value'] = "We truly value your business and look forward to serving you again soon!"
df_test.groupby(['Overall_order_experience', 'Test Experience'])['Business value'].nunique()

Overall_order_experience  Test Experience
Bad                       $10 off            1
                          $5 off             1
                          Control            1
                          No offer           1
Good                      $10 off            1
                          $5 off             1
                          Control            1
                          No offer           1
Name: Business value, dtype: int64

In [109]:
#Assigning dynamic text relating to household loyalty for the email template
df_test['HHS_Loyalty'] = ""
df_test.loc[df_test['fresh_pass_hhs'] == True, 'HHS_Loyalty'] = "and for your support as a FreshPass subscriber!"
df_test.loc[(df_test['fresh_pass_hhs'] == False) & 
            (df_test['Ecomm txns in last 6 months'] == '5+ txns'), 'HHS_Loyalty'] = "and for your continued support as a loyal online customer."
df_test.loc[(df_test['fresh_pass_hhs'] == False) & 
            (df_test['Ecomm txns in last 6 months'] == '1-4 txns'), 'HHS_Loyalty'] = "and for your continued support."
df_test.loc[(df_test['fresh_pass_hhs'] == False) & 
            (df_test['Customer_Segment'] == 'BNC to eComm'), 'HHS_Loyalty'] = "and for trying us online for the first time!"
df_test.loc[(df_test['fresh_pass_hhs'] == False) & 
            (df_test['Customer_Segment'] == 'BNC to ACI'), 'HHS_Loyalty'] = "and for trying us online for the first time!"
df_test.groupby(['fresh_pass_hhs','Customer_Segment', 'Ecomm txns in last 6 months'])['order_id'].count()

fresh_pass_hhs  Customer_Segment            Ecomm txns in last 6 months
False           Acquired prior to 6 months  0 txns                          2003
                                            1-4 txns                        7101
                                            5+ txns                        35867
                Acquired within 6 months    1-4 txns                        4787
                                            5+ txns                         4846
                BNC to ACI                  0 txns                          1075
                BNC to eComm                0 txns                          3130
True            Acquired prior to 6 months  0 txns                            74
                                            1-4 txns                         439
                                            5+ txns                         9162
                Acquired within 6 months    1-4 txns                         567
                                     

In [110]:
#Adding offers related data for the emails and allocation
df_week_offer = df_offers[pd.to_datetime(df_offers["offer_start_date"]) == (datetime.today() + timedelta(days=1)).strftime('%m/%d/%Y')]
#df_week_offer = df_offers[pd.to_datetime(df_offers["offer_start_date"]) == (datetime.today() + timedelta(days=0)).strftime('%m/%d/%Y')]

In [111]:
df_week_offer

,offer_id,image_id,price_text,headline_1,offer_start_date,offer_end_date,display_end_date,test_experience


In [112]:
df_int = pd.merge(df_test, df_week_offer, how='left', left_on='Test Experience', right_on='test_experience')
df_test = pd.merge(df_test, df_week_offer, how='left', left_on='Test Experience', right_on='test_experience')
df_int[df_int['Test Experience'] != 'Control']

,index,order_status_nm,version_id,order_id,order_dt,channel_type,delivery_type,svclvl_type,stype,txn_id,...,Business value,HHS_Loyalty,offer_id,image_id,price_text,headline_1,offer_start_date,offer_end_date,display_end_date,test_experience
0,2940,COMPLETED,1,78822321,2024-03-04,APP,DUG,STANDARD,DUG,6897249595,...,"As our way of saying thanks, redeem this offer...",and for trying us online for the first time!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,45408,COMPLETED,1,83964465,2024-03-04,SITE,DELIVERY,EXPRESS,2HR,6895185017,...,"As our way of saying thanks, redeem this offer...",and for trying us online for the first time!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,51699,COMPLETED,2,83911638,2024-03-03,APP,DUG,STANDARD,DUG,6895707709,...,"As our way of saying thanks, redeem this offer...",and for trying us online for the first time!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11640,COMPLETED,1,84011534,2024-03-04,APP,DUG,EXPRESS,DUG,6898103702,...,"As our way of saying thanks, redeem this offer...",and for trying us online for the first time!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,11,COMPLETED,1,83953188,2024-03-03,APP,DUG,STANDARD,DUG,6895613950,...,"As our way of saying thanks, redeem this offer...",and for trying us online for the first time!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70499,62423,COMPLETED,1,83945433,2024-03-03,APP,DUG,STANDARD,DUG,6899181502,...,"We value your business, and know we can do bet...",and for trying us online for the first time!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70500,71124,COMPLETED,1,84015405,2024-03-04,APP,DUG,STANDARD,DUG,6900054811,...,"We value your business, and know we can do bet...",and for trying us online for the first time!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70501,71883,COMPLETED,1,84002082,2024-03-04,APP,DUG,STANDARD,DUG,6899979181,...,"We value your business, and know we can do bet...",and for trying us online for the first time!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70502,74818,COMPLETED,1,84022323,2024-03-04,APP,DELIVERY,EXPRESS,2HR,6899645697,...,"We value your business, and know we can do bet...",and for trying us online for the first time!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:

df_upload_analytics = df_test[['household_id', 'order_id', 'txn_dte', 'delivery_type','oos', 'subs', 'OOS_percent', 'Pre_Subs_OOS_Per',
        'comments', 'refund_reasons', 'deliv_ts', 'wait_time_in_mins', 'till_by_time',
        'slot_start', 'slot_end', 'SCORE_PreSub_OOS_per', 'SCORE_OOS_per',
        'SCORE_NPS_Order_Accuracy', 'SCORE_NPS_Substitution_Satisfaction',
        'Order_Accuracy_NPS', 'Order_Accuracy_Operational', 'Order_Accuracy_Final',
        'Order_Accuracy_Complaints', 'Order_Accuracy_Refunds', 'Order_Accuracy_Final_inc_C_R',
        'SCORE_OTD', 'SCORE_OTH', 'SCORE_Tilling',
        'SCORE_NPS_Order_Delivery','Order_Delivery_Operational', 'Order_Delivery_Final',
        'Order_Delivery_Complaints', 'Order_Delivery_Refunds', 'Order_Delivery_Final_inc_C_R',
        'SCORE_Refunds', 'SCORE_Complaints', 'SCORE_NPS_Product_Quality','SCORE_NPS_Product_Freshness',
        'Order_Condition_NPS','Order_Condition_Operational', 'Order_Condition_Final', 
        'offer_id', 'price_text', 'banner', 'store_id', 'Theme', 
        'offer_start_date', 'offer_end_date', 'Customer_Segment','Ecomm txns in last 6 months' ,
        'Weeks since last ecomm txn' , 'urbanicity' ,'HHS_Loyalty', 'Product Availability', 'Product Availability Score',
        'Product Availability Alt Text', 'Order Delivery','Order Delivery Score','Order Delivery Alt Text',
        'Order Condition','Order Condition Score','Order Condition Alt Text','Order Summary','Business value',
        'NPS link', 'Test Experience']]
df_upload_analytics.columns = ['HHS_ID', 'order_id', 'txn_dte', 'delivery_type','oos', 'subs', 'OOS_percent', 'Pre_Subs_OOS_Per',
                               'comments', 'refund_reasons', 'deliv_ts', 'wait_time_in_mins', 'till_by_time',
                               'slot_start', 'slot_end', 'SCORE_PreSub_OOS_per', 'SCORE_OOS_per',
                               'SCORE_NPS_Order_Accuracy', 'SCORE_NPS_Substitution_Satisfaction',
                               'Order_Accuracy_NPS', 'Order_Accuracy_Operational', 'Order_Accuracy_Final',
                               'Order_Accuracy_Complaints', 'Order_Accuracy_Refunds', 'Order_Accuracy_Final_inc_C_R',
                               'SCORE_OTD', 'SCORE_OTH', 'SCORE_Tilling',
                               'SCORE_NPS_Order_Delivery','Order_Delivery_Operational', 'Order_Delivery_Final',
                               'Order_Delivery_Complaints', 'Order_Delivery_Refunds', 'Order_Delivery_Final_inc_C_R',
                               'SCORE_Refunds', 'SCORE_Complaints', 'SCORE_NPS_Product_Quality','SCORE_NPS_Product_Freshness',
                               'Order_Condition_NPS','Order_Condition_Operational', 'Order_Condition_Final', 
                               'offer_code', 'offer_value', 'banner', 'store_id', 'Theme', 
                               'offer_start_date', 'offer_end_date', 'Customer_Segment','Ecomm_txns_in_last_6_months' ,
                               'Weeks_since_last_ecomm_txn' , 'urbanicity' ,'HHS_Loyalty', 'Product_Availability', 'Product_Availability_Score',
                               'Product_Availability_Alt_Text', 'Order_Delivery','Order_Delivery_Score','Order_Delivery_Alt_Text',
                               'Order_Condition','Order_Condition_Score','Order_Condition_Alt_Text','Order_Summary','Business_value',
                               'NPS_link', 'Test_Experience']
df_int_2 = df_int[df_int['Test Experience'] != 'Control']
df_final_upload = df_int_2[['household_id', 'offer_id', 'price_text', 'order_id', 'txn_dte', 'banner', 'store_id', 'Theme', 
        'offer_start_date', 'offer_end_date', 'HHS_Loyalty', 'Product Availability', 'Product Availability Score',
       'Product Availability Alt Text', 'Order Delivery','Order Delivery Score','Order Delivery Alt Text',
        'Order Condition','Order Condition Score','Order Condition Alt Text','Order Summary','Business value',
        'NPS link']]
df_final_upload.columns = ['HHS_ID','Offer Code','Offer Value','ORDER_ID','Order fulfilled date','Banner','STORE_ID',
                          'Theme','Offer Start Date','Offer End Date','HHS_Loyalty','Product Availability',
                          'Product Availability Score','Product Availability Alt Text','Order Delivery','Order Delivery Score',
                          'Order Delivery Alt Text','Order Condition','Order Condition Score','Order Condition Alt Text','Order Summary',
                          'Business value','NPS link']

In [114]:
#Loading final output file to EDM
# conn = SFConnection()
# conn.add_name("nacke08")
# conn.test_conn()
df_final_upload.columns = df_final_upload.columns.str.upper()
df_final_upload['UPDATE_DATE'] = dt.datetime.now().strftime("%Y-%m-%d %H:%M")
df_final_upload['ORDER FULFILLED DATE'] = pd.to_datetime(df_final_upload['ORDER FULFILLED DATE'])
df_final_upload['ORDER FULFILLED DATE'] = df_final_upload['ORDER FULFILLED DATE'].dt.strftime("%m/%d/%Y")
#conn.load_dataframe_REPLACE(df_final_upload, 'esa_proactive_communication_output')

C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\3322443596.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_upload['UPDATE_DATE'] = dt.datetime.now().strftime("%Y-%m-%d %H:%M")
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\3322443596.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_upload['ORDER FULFILLED DATE'] = pd.to_datetime(df_final_upload['ORDER FULFILLED DATE'])
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\3322443596.py:8: SettingWithCopyWarning: 
A value is t

In [115]:
#Loading data to be used for analysis to EDM
df_upload_analytics['EMAIL_DATE'] = (datetime.today() + timedelta(days=1)).strftime('%m/%d/%Y')
df_upload_analytics['UPDATE_DATE'] = datetime.today().strftime('%Y-%m-%d %H:%M')
df_upload_analytics.columns = df_upload_analytics.columns.str.upper()
print(df_upload_analytics)
#df_upload_analytics.to_excel(r'C:\Users\nacke08\OneDrive - Safeway, Inc\Documents\Projects\Ecommerce\FY23\ADHOC\EXCEL\PROACTIVE_TESTING_01_30_24.xlsx', index = False)
df_upload_analytics.to_excel('TESTING_PROACTIVE_E2E_3_6_PROD.xlsx')
#conn.load_dataframe_APPEND(df_upload_analytics, 'esa_proactive_communication_analysis_output_ott_mod')
conn.close_conn()


C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2922547777.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_upload_analytics['EMAIL_DATE'] = (datetime.today() + timedelta(days=1)).strftime('%m/%d/%Y')
C:\Users\nacke08\AppData\Local\Temp\ipykernel_24700\2922547777.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_upload_analytics['UPDATE_DATE'] = datetime.today().strftime('%Y-%m-%d %H:%M')


             HHS_ID  ORDER_ID     TXN_DTE DELIVERY_TYPE  OOS  SUBS  \
0      845037810035  78822321  2024-03-04           DUG  0.0   0.0   
1      816031261207  83964465  2024-03-04      DELIVERY  0.0   0.0   
2      990017076767  83911638  2024-03-04           DUG  1.0   0.0   
3      990073545295  83855549  2024-03-04      DELIVERY  0.0   1.0   
4      787019740247  84011534  2024-03-04           DUG  0.0   0.0   
...             ...       ...         ...           ...  ...   ...   
70499  787027414412  83945433  2024-03-04           DUG  0.0   1.0   
70500  816029240716  84015405  2024-03-04           DUG  4.0   1.0   
70501  290087283774  84002082  2024-03-04           DUG  0.0   5.0   
70502  250001277555  84022323  2024-03-04      DELIVERY  0.0   2.0   
70503  990051092880  83934416  2024-03-04           DUG  0.0   3.0   

       OOS_PERCENT  PRE_SUBS_OOS_PER COMMENTS REFUND_REASONS  ...  \
0         0.000000          0.000000     None           None  ...   
1         0.000000   